In [17]:
# Load libraries (kernel: napari-sparrow)

import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas

## Related to single-cell
from anndata import AnnData
import scanpy as sc
import squidpy as sq

import napari

# Prepare data

## Load data

In [18]:
# Load anndata and polygons
adata      = sc.read("/Users/yujike/Desktop/Molecular_Plant_2023/Manuscript/vincent_very-crude-visualization/v1/adata.h5ad")
geop       = geopandas.read_file("/Users/yujike/Desktop/Molecular_Plant_2023/Manuscript/vincent_very-crude-visualization/v1/adata.geojson", index_col = 0)
geop.index = geop['index']
adata.obsm['polygons'] = geop
adata.obsm['polygons']['linewidth'] = 1

del geop

## Select cells

In [20]:
# Redefine the filtering function (the original does not work)
from typing import List, Optional, Tuple
def filter_on_size(
    adata: AnnData, min_size: int = 100, max_size: int = 100000
) -> Tuple[AnnData, int]:
    """Returns a tuple with the AnnData object and the number of filtered cells.

    All cells outside of the min and max size range are removed.
    If the distance between the location of the transcript and the center of the polygon is large, the cell is deleted.
    """

    start = adata.shape[0]

    # Calculate center of the cell and distance between transcript and polygon center
    adata.obsm["polygons"]["X"] = adata.obsm["polygons"].centroid.x
    adata.obsm["polygons"]["Y"] = adata.obsm["polygons"].centroid.y
    adata.obs["distance"] = np.sqrt(
        np.square(adata.obsm["polygons"]["X"] - adata.obsm["spatial"].iloc[:, 0])
        + np.square(adata.obsm["polygons"]["Y"] - adata.obsm["spatial"].iloc[:, 1])
    )

    # Filter cells based on size and distance
    #adata = adata[adata.obs["nucleusSize"] < max_size, :]
    #adata = adata[adata.obs["nucleusSize"] > min_size, :]
    adata=adata[adata.obsm['polygons'].area>min_size,:]
    adata.obsm["polygons"] = geopandas.GeoDataFrame(
        adata.obsm["polygons"], geometry=adata.obsm["polygons"].geometry
    )
    adata=adata[adata.obsm['polygons'].area<max_size,:]

    adata = adata[adata.obs["distance"] < 70, :]
    adata.obsm["polygons"] = geopandas.GeoDataFrame(
        adata.obsm["polygons"], geometry=adata.obsm["polygons"].geometry
    )
    filtered = start - adata.shape[0]

    return adata, filtered

In [21]:
# Filter
adata, filtered = filter_on_size(adata, min_size = 200)

/var/folders/ch/nyzh10996mzbm4r4pj4m0qd00000gn/T/ipykernel_26370/3350284418.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  adata.obsm["polygons"]["X"] = adata.obsm["polygons"].centroid.x
/var/folders/ch/nyzh10996mzbm4r4pj4m0qd00000gn/T/ipykernel_26370/3350284418.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  adata.obsm["polygons"]["Y"] = adata.obsm["polygons"].centroid.y
/var/folders/ch/nyzh10996mzbm4r4pj4m0qd00000gn/T/ipykernel_26370/3350284418.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  adata=adata[adata.obsm['polygons'].area>min_size,:]
/var/folders/c

## Prepare for Napari

In [22]:
# Function to select the representative polygons and add them as shape
def prepare_napari_plot(adata, cutoff_shapes = 0.5):
    temp                 = adata.obsm['polygons'].explode()
    temp['percent_area'] = temp.area / adata.obsm['polygons'].area
    temp                 = temp.sort_values(by = 'percent_area', ascending = False)
    temp.index           = temp.index.droplevel(1)
    temp                 = temp[~temp.index.duplicated(keep = 'first')]
    temp                 = temp.reindex(adata.obs.index)

    adata = adata[temp.percent_area > cutoff_shapes, :]
    temp2 = temp[temp.percent_area > cutoff_shapes]
    adata.obsm['napari_plot'] = temp2[['X','Y','geometry','percent_area']]

    print('In total, '+str(len(temp)-len(temp2))+' shapes were filtered out because they were too split up.')
    return adata 

In [23]:
adata = prepare_napari_plot(adata)

/var/folders/ch/nyzh10996mzbm4r4pj4m0qd00000gn/T/ipykernel_26370/3313942367.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  temp                 = adata.obsm['polygons'].explode()
/var/folders/ch/nyzh10996mzbm4r4pj4m0qd00000gn/T/ipykernel_26370/3313942367.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  temp['percent_area'] = temp.area / adata.obsm['polygons'].area


In total, 7 shapes were filtered out because they were too split up.


# Visualize in Napari

## Initialize Napari

In [24]:
# Function to add a layer to napari
def add_napari_layer(adata, column = None, name = None, celltype = None, colormap = 'inferno'):
    """If you want a subset, just take a subset of the anndata before adding this. Don't use the function to change color, only to add an extra layer."""
    if celltype: 
        adata=adata[adata.obs['maxScores']==celltype,:]
        if not name:
            name=celltype
    coords_to_plot=[]
    to_color=None
    adata.obsm['napari_plot']=geopandas.GeoDataFrame(adata.obsm['napari_plot'],geometry=adata.obsm['napari_plot']['geometry'])
    if column is not None:
        if column + "_colors" in adata.uns:
            color_dict=dict(zip(adata.obs[column].cat.categories.values,adata.uns[column + "_colors"]))
            to_color=adata.obs[column].map(color_dict)
            print('colors from adata used')
    for i in adata.obsm['napari_plot'].geometry.exterior.simplify(tolerance=2):
        coords_to_plot.append(np.array(i.coords)[:,[1,0]]) 
    Columns=dict(zip(adata.var.index.values,adata.X.T))
    Columns.update(adata.obs.to_dict('list'))
    if not name:
        name=column
    if name in viewer.layers:
        viewer.layers[name].features=Columns
        if to_color:
            print('layer reused')
            viewer.layers[name].face_color=to_color.values
        if column in viewer.layers[name].features:
            print('layer reused')
            viewer.layers[name].face_color=column
            viewer.layers[name].face_colormap =colormap
    elif to_color is not None:
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=to_color.values.astype(str),opacity=0.7,edge_width=3,edge_color='white',features=Columns)
    elif column in Columns.keys():
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=column,opacity=0.7,edge_width=3,edge_color='white',features=Columns,face_colormap=colormap)
    elif column==None:
        viewer.add_shapes(coords_to_plot,name=name,shape_type='polygon',face_color=colormap,opacity=0.7,edge_width=3,edge_color='white',features=Columns)

In [25]:
# Initialize napari with image
I = adata.uns['spatial']['spatial_transcriptomics']['images']['hires']
#viewer = napari.view_image(Id, rgb=False)
viewer = napari.Viewer()
max_contrast_limit = np.max(I)
# max_contrast_limit = np.iinfo(img.dtype).max
viewer.add_image(I, contrast_limits=[0, max_contrast_limit])  
#viewer.add_shapes(coords_to_plot,name='DAPI',shape_type='polygon',face_color='blue',opacity=0.5,edge_width=3,edge_color='white')

/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Image layer 'I' at 0x7fb0712b5070>

## Add segments

In [26]:
add_napari_layer(adata)

/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/layers/utils/color_transformations.py:42: UserWarning: The provided face_color parameter contained illegal values, resetting all face_color values to white.
  warnings.warn(


In [27]:
add_napari_layer(adata, column = 'TraesCS3D03G1015700') # yes




In [50]:
# how many genes in tae_2 cortex showed expected expression 
add_napari_layer(adata, column = 'TraesCS7B03G0820900') # yes
add_napari_layer(adata, column = 'TraesCS2B03G1383800LC') # yes
add_napari_layer(adata, column = 'TraesCS4D03G0728500') # no
add_napari_layer(adata, column = 'TraesCS1D03G0534600') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0676300') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0534700') # no
add_napari_layer(adata, column = 'TraesCS2A03G0988000') # no
add_napari_layer(adata, column = 'TraesCS4B03G0814900') # yes
add_napari_layer(adata, column = 'TraesCS3B03G1121400') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0768600') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0646400') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0817100') # no
add_napari_layer(adata, column = 'TraesCS1A03G0554700') # yes
add_napari_layer(adata, column = 'TraesCS2B03G1096500') # no
add_napari_layer(adata, column = 'TraesCS3D03G0911000') # yes
add_napari_layer(adata, column = 'TraesCS2D03G0925700') # yes
add_napari_layer(adata, column = 'TraesCS4B03G0814800') # no
add_napari_layer(adata, column = 'TraesCS3A03G0974500') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0787300') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0787400') # yes

#  

In [60]:
# how many genes in tae_3 meristems showed expected expression 
add_napari_layer(adata, column = 'TraesCS2B03G1047100') # yes
add_napari_layer(adata, column = 'TraesCS1B03G1035400') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0723300') # yes
add_napari_layer(adata, column = 'TraesCS6A03G0197200') # yes
add_napari_layer(adata, column = 'TraesCS4B03G0567900') # yes
add_napari_layer(adata, column = 'TraesCS3A03G0316900') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0080700') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0687000') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0852900') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0090800') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0080500') # yes
add_napari_layer(adata, column = 'TraesCS2A03G0948500') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0109700') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0853100') # yes
add_napari_layer(adata, column = 'TraesCS7D03G0261200') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0825800') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0761600') # yes
add_napari_layer(adata, column = 'TraesCS6A03G0141200') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0773600') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0730400') # yes


In [52]:
# how many genes in tae_4 provascular cells showed expected expression pattern
add_napari_layer(adata, column = 'TraesCS5D03G0346300') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0622300') # no
add_napari_layer(adata, column = 'TraesCS1B03G0050600') # yes
add_napari_layer(adata, column = 'TraesCS6D03G0550100LC') # no
add_napari_layer(adata, column = 'TraesCS3B03G0036500') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0532100') # no
add_napari_layer(adata, column = 'TraesCS5B03G0678400LC') # no 
add_napari_layer(adata, column = 'TraesCS2B03G1307900') # no
add_napari_layer(adata, column = 'TraesCS3D03G0132200') # no
add_napari_layer(adata, column = 'TraesCS6D03G0573300') # no
add_napari_layer(adata, column = 'TraesCS4D03G0562400') # no
add_napari_layer(adata, column = 'TraesCS1B03G0053600') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0107600') # no
add_napari_layer(adata, column = 'TraesCS7D03G1240300') # no
add_napari_layer(adata, column = 'TraesCS2A03G0412800') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0051800') # yes
add_napari_layer(adata, column = 'TraesCS7A03G0475200') # no
add_napari_layer(adata, column = 'TraesCS6A03G0661700LC') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0013700') # yes
add_napari_layer(adata, column = 'TraesCS3A03G0016400') # no


In [59]:
# how many genes in tae_7 epidermis showed expected expression pattern
add_napari_layer(adata, column = 'TraesCS2D03G0324500') # yes
add_napari_layer(adata, column = 'TraesCS7A03G0992500LC') # no
add_napari_layer(adata, column = 'TraesCS2D03G0324400') # yes
add_napari_layer(adata, column = 'TraesCS2A03G0309200') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0426900') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0426800') # yes
add_napari_layer(adata, column = 'TraesCS2A03G0309100LC') # yes
add_napari_layer(adata, column = 'TraesCS7B03G1286700') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0426600LC') # yes
add_napari_layer(adata, column = 'TraesCS5B03G1313300') # yes
add_napari_layer(adata, column = 'TraesCS2B03G1540400') # yes
add_napari_layer(adata, column = 'TraesCS6B03G0242600') # yes
add_napari_layer(adata, column = 'TraesCS3D03G1015700') # yes
add_napari_layer(adata, column = 'TraesCS4A03G0065400') # no
add_napari_layer(adata, column = 'TraesCS1D03G0362900') # yes
add_napari_layer(adata, column = 'TraesCS7B03G0829700LC') # yes
add_napari_layer(adata, column = 'TraesCS5D03G1173000') # yes
add_napari_layer(adata, column = 'TraesCS3A03G0103100') # yes
add_napari_layer(adata, column = 'TraesCS7A03G0992200LC') # yes
add_napari_layer(adata, column = 'TraesCS2B03G1534500') # yes


In [58]:
# how many genes in tae_9 endodermis showed expected expression pattern 
add_napari_layer(adata, column = 'TraesCS3A03G0429700') # no
add_napari_layer(adata, column = 'TraesCS7D03G0401600') # no
add_napari_layer(adata, column = 'TraesCS7B03G0218800') # no
add_napari_layer(adata, column = 'TraesCS2A03G0749000') # no
add_napari_layer(adata, column = 'TraesCS3D03G0402600') # yes
add_napari_layer(adata, column = 'TraesCS3B03G0898100') # no
add_napari_layer(adata, column = 'TraesCS3B03G0508400') # no
add_napari_layer(adata, column = 'TraesCS6D03G0764800') # no
add_napari_layer(adata, column = 'TraesCS2D03G0692900') # no
add_napari_layer(adata, column = 'TraesCS5A03G0962700') # no
add_napari_layer(adata, column = 'TraesCS6A03G0889900') # no
add_napari_layer(adata, column = 'TraesCS2B03G0423100') # no
add_napari_layer(adata, column = 'TraesCS3D03G0720200') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0105400') # no
add_napari_layer(adata, column = 'TraesCS6A03G0661900') # no
add_napari_layer(adata, column = 'TraesCS5D03G0492700') # yes
add_napari_layer(adata, column = 'TraesCS2D03G0321300') # yes
add_napari_layer(adata, column = 'TraesCS5B03G0538700') # yes
add_napari_layer(adata, column = 'TraesCS7A03G1365400') # no
add_napari_layer(adata, column = 'TraesCS1B03G0907000') # no



In [57]:

# how many genes in tae_10 xylem showed expected expression pattern

add_napari_layer(adata, column = 'TraesCS5D03G0377000') # yes
add_napari_layer(adata, column = 'TraesCS5A03G0388600') # yes
add_napari_layer(adata, column = 'TraesCS5B03G0387000') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0049400') # no
add_napari_layer(adata, column = 'TraesCS1A03G0062800') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0484900') # no
add_napari_layer(adata, column = 'TraesCS7B03G0805100') # no
add_napari_layer(adata, column = 'TraesCS3B03G1344800') # no
add_napari_layer(adata, column = 'TraesCS6D03G0082600') # no
add_napari_layer(adata, column = 'TraesCSU03G0020100') # no
add_napari_layer(adata, column = 'TraesCS2A03G0730800') # no
add_napari_layer(adata, column = 'TraesCS6D03G0654300') # no
add_napari_layer(adata, column = 'TraesCS7A03G0969300') # no
add_napari_layer(adata, column = 'TraesCS3B03G0012900') # no
add_napari_layer(adata, column = 'TraesCS3D03G0008300') # no
add_napari_layer(adata, column = 'TraesCS7A03G0446200') # no
add_napari_layer(adata, column = 'TraesCS2B03G0806600') # no
add_napari_layer(adata, column = 'TraesCS6A03G0080100') # no
add_napari_layer(adata, column = 'TraesCSU03G0019900') # no
add_napari_layer(adata, column = 'TraesCS5A03G0570800') # no



In [61]:
# how many genes in tae_12 dividing cells showed expected expression pattern

add_napari_layer(adata, column = 'TraesCS2D03G0803000') # yes
add_napari_layer(adata, column = 'TraesCS3B03G0839200LC') # yes
add_napari_layer(adata, column = 'TraesCS2A03G0863000') # yes
add_napari_layer(adata, column = 'TraesCS2B03G0955500') # yes
add_napari_layer(adata, column = 'TraesCS4D03G0742700') # yes
add_napari_layer(adata, column = 'TraesCS3A03G0796600') # yes
add_napari_layer(adata, column = 'TraesCS3B03G1202600') # yes
add_napari_layer(adata, column = 'TraesCS3A03G0935100') # yes
add_napari_layer(adata, column = 'TraesCS2D03G0129500') # yes
add_napari_layer(adata, column = 'TraesCS1B03G0616100') # yes
add_napari_layer(adata, column = 'TraesCS1A03G0526900') # yes
add_napari_layer(adata, column = 'TraesCS4B03G0833500') # yes
add_napari_layer(adata, column = 'TraesCS3B03G0918400') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0735900') # yes
add_napari_layer(adata, column = 'TraesCS1D03G0508400') # yes
add_napari_layer(adata, column = 'TraesCS7B03G1269700') # yes
add_napari_layer(adata, column = 'TraesCS7D03G1262100') # yes
add_napari_layer(adata, column = 'TraesCS2A03G0132400') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0969300') # yes
add_napari_layer(adata, column = 'TraesCS7A03G1335600') # yes


In [62]:
# how many genes in tae_13 phloem showed expected expression pattern

add_napari_layer(adata, column = 'TraesCS2B03G0108200') # yes
add_napari_layer(adata, column = 'TraesCS2D03G0068900') # yes
add_napari_layer(adata, column = 'TraesCS3B03G0900500') # no
add_napari_layer(adata, column = 'TraesCS2A03G0073400') # yes
add_napari_layer(adata, column = 'TraesCS3D03G0721900') # no
add_napari_layer(adata, column = 'TraesCS3A03G0782300') # no
add_napari_layer(adata, column = 'TraesCS1D03G0884600') # no
add_napari_layer(adata, column = 'TraesCS3D03G0599200') # no
add_napari_layer(adata, column = 'TraesCS6A03G0666600') # no
add_napari_layer(adata, column = 'TraesCS3A03G0654700') # no
add_napari_layer(adata, column = 'TraesCS1D03G0605000') # no
add_napari_layer(adata, column = 'TraesCS1B03G0733300') # no
add_napari_layer(adata, column = 'TraesCS1A03G0647000') # no
add_napari_layer(adata, column = 'TraesCS1A03G0906900') # no
add_napari_layer(adata, column = 'TraesCS3B03G0742300') # no
add_napari_layer(adata, column = 'TraesCS6A03G0202900') # yes
add_napari_layer(adata, column = 'TraesCS4D03G0020800') # no
add_napari_layer(adata, column = 'TraesCS7D03G0500500') # yes
add_napari_layer(adata, column = 'TraesCS5D03G0557600') # no
add_napari_layer(adata, column = 'TraesCS3B03G0486000') # no


In [63]:
# how many genes in tae_14 root cap showed expected expression pattern

add_napari_layer(adata, column = 'TraesCS7D03G0226600') #
add_napari_layer(adata, column = 'TraesCS3D03G0697200') #
add_napari_layer(adata, column = 'TraesCS2D03G0971100') #
add_napari_layer(adata, column = 'TraesCS3D03G0382600') #
add_napari_layer(adata, column = 'TraesCS2A03G1032300') #
add_napari_layer(adata, column = 'TraesCS3B03G0864600') #
add_napari_layer(adata, column = 'TraesCS1D03G0043400') #
add_napari_layer(adata, column = 'TraesCS3D03G1163600') #
add_napari_layer(adata, column = 'TraesCS3B03G1268400') #
add_napari_layer(adata, column = 'TraesCS3D03G1163500') #
add_napari_layer(adata, column = 'TraesCS3A03G0756500') #
add_napari_layer(adata, column = 'TraesCS1A03G0053300') #
add_napari_layer(adata, column = 'TraesCS3B03G1464600') #
add_napari_layer(adata, column = 'TraesCS1B03G0058000') #
add_napari_layer(adata, column = 'TraesCS3B03G1464500') #
add_napari_layer(adata, column = 'TraesCS2B03G1153200') #
add_napari_layer(adata, column = 'TraesCS2D03G0971700') #
add_napari_layer(adata, column = 'TraesCS3A03G1224300') #
add_napari_layer(adata, column = 'TraesCS2D03G0971800') #
add_napari_layer(adata, column = 'TraesCS3D03G1163700') #


In [10]:
# Phloem conserved marker
add_napari_layer(adata, column = 'TraesCS7A03G0757500') #
add_napari_layer(adata, column = 'TraesCS7D03G0728800') #
add_napari_layer(adata, column = 'TraesCS7B03G0588000') #




2024-06-12 20:20:17.390 python[1445:17999] +[CATransaction synchronize] called within transaction
2024-06-12 20:20:17.528 python[1445:17999] +[CATransaction synchronize] called within transaction


In [36]:

add_napari_layer(adata, column = 'TraesCS3D03G1031300') #



In [40]:
add_napari_layer(adata, column = 'TraesCS1A03G0554700') #


2024-06-14 13:32:36.129 python[1445:17999] +[CATransaction synchronize] called within transaction
2024-06-14 13:32:36.262 python[1445:17999] +[CATransaction synchronize] called within transaction


In [39]:
add_napari_layer(adata, column = 'TraesCS4D03G0434100') #
add_napari_layer(adata, column = 'TraesCS4A03G0280400') #
add_napari_layer(adata, column = 'TraesCS4B03G0486500') #



2024-06-14 12:43:49.955 python[1445:17999] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (1.38 secs).


In [38]:

add_napari_layer(adata, column = 'TraesCS4A03G0065400') #



In [33]:
add_napari_layer(adata, column = 'TraesCS6B03G1186600') #


In [15]:

add_napari_layer(adata, column = 'TraesCS4B03G0238600') #
add_napari_layer(adata, column = 'TraesCS4D03G0201600') #

	





In [27]:
#Figure 2 Epidermis
add_napari_layer(adata, column = 'TraesCS2D03G0324400') #
add_napari_layer(adata, column = 'TraesCS2D03G0324500') #

In [29]:
#Figure 2 Cortex
add_napari_layer(adata, column = 'TraesCS1D03G0787400') #
add_napari_layer(adata, column = 'TraesCS3A03G0974500') #
add_napari_layer(adata, column = 'TraesCS3D03G0911000') #
add_napari_layer(adata, column = 'TraesCS1B03G0646400') #
add_napari_layer(adata, column = 'TraesCS3B03G1121400') #
add_napari_layer(adata, column = 'TraesCS3B03G1120300') #
add_napari_layer(adata, column = 'TraesCS2B03G1501600') #
add_napari_layer(adata, column = 'TraesCS2B03G1383800LC') #












2024-06-11 21:11:19.992 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:11:20.142 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:15:14.948 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:15:15.106 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:15:19.340 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:15:21.098 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:22:17.683 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:22:17.803 python[4516:91078] +[CATransaction synchronize] called within transaction


In [30]:
#Figure 2 Xylem
add_napari_layer(adata, column = 'TraesCS5D03G0377000') #
add_napari_layer(adata, column = 'TraesCS5A03G0388600') #

2024-06-11 21:49:54.000 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-11 21:49:54.109 python[4516:91078] +[CATransaction synchronize] called within transaction


In [31]:
#Figure 2 Phloem
add_napari_layer(adata, column = 'TraesCS2D03G0068900') #
add_napari_layer(adata, column = 'TraesCS6A03G0202900') #
add_napari_layer(adata, column = 'TraesCS5A03G0536500') #
add_napari_layer(adata, column = 'TraesCS5A03G1176600LC') #


2024-06-12 08:46:20.407 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 08:46:21.061 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 08:46:24.873 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 08:52:36.902 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 08:53:03.672 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 08:53:03.831 python[4516:91078] +[CATransaction synchronize] called within transaction


In [33]:
#Figure 2 Provascular cells

add_napari_layer(adata, column = 'TraesCS5A03G0363500') #
add_napari_layer(adata, column = 'TraesCS5B03G0678500LC') #
add_napari_layer(adata, column = 'TraesCS5B03G0359300') #
add_napari_layer(adata, column = 'TraesCS5D03G0346300') #
add_napari_layer(adata, column = 'TraesCS1B03G0622300') #
add_napari_layer(adata, column = 'TraesCS1B03G0050600') #
add_napari_layer(adata, column = 'TraesCS6D03G0550100LC') #
add_napari_layer(adata, column = 'TraesCS3B03G0036500') #
add_napari_layer(adata, column = 'TraesCS1A03G0532100') #
add_napari_layer(adata, column = 'TraesCS5B03G0678400LC') #
add_napari_layer(adata, column = 'TraesCS2B03G1307900') #
add_napari_layer(adata, column = 'TraesCS3D03G0132200') #
add_napari_layer(adata, column = 'TraesCS6D03G0573300') #
add_napari_layer(adata, column = 'TraesCS4D03G0562400') #
add_napari_layer(adata, column = 'TraesCS1B03G0053600') #
add_napari_layer(adata, column = 'TraesCS2B03G0107600') #
add_napari_layer(adata, column = 'TraesCS7D03G1240300') #
add_napari_layer(adata, column = 'TraesCS2A03G0412800') #
add_napari_layer(adata, column = 'TraesCS1B03G0051800') #
add_napari_layer(adata, column = 'TraesCS7A03G0475200') #
add_napari_layer(adata, column = 'TraesCS6A03G0661700LC') #
add_napari_layer(adata, column = 'TraesCS3D03G0013700') #
add_napari_layer(adata, column = 'TraesCS3A03G0016400') #













2024-06-12 10:12:41.068 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 10:12:41.204 python[4516:91078] +[CATransaction synchronize] called within transaction


In [34]:
# Figure 2 Dividing cell
add_napari_layer(adata, column = 'TraesCS2D03G0803000') #
add_napari_layer(adata, column = 'TraesCS3B03G0839200LC') #
add_napari_layer(adata, column = 'TraesCS2A03G0863000') #
add_napari_layer(adata, column = 'TraesCS2B03G0955500') #
add_napari_layer(adata, column = 'TraesCS4D03G0742700') #
add_napari_layer(adata, column = 'TraesCS3A03G0796600') #


2024-06-12 10:33:50.050 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 10:33:50.161 python[4516:91078] +[CATransaction synchronize] called within transaction


In [12]:
#Conserved cortex marker

add_napari_layer(adata, column = 'TraesCS1A03G0554700') #
add_napari_layer(adata, column = 'TraesCS1B03G0646400') #
add_napari_layer(adata, column = 'TraesCS1D03G0534600') #




2024-06-09 15:49:29.798 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 15:49:29.930 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 15:49:37.257 python[4516:91078] -[NSNull length]: unrecognized selector sent to instance 0x7ff84a39cb40
2024-06-09 15:49:37.259 python[4516:91078] -[NSNull length]: unrecognized selector sent to instance 0x7ff84a39cb40
2024-06-09 15:49:39.870 python[4516:91078] -[NSNull length]: unrecognized selector sent to instance 0x7ff84a39cb40
2024-06-09 15:49:39.912 python[4516:91078] -[NSNull length]: unrecognized selector sent to instance 0x7ff84a39cb40
2024-06-09 15:50:11.060 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 15:50:25.428 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 15:51:03.326 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 15:51:03.409 python[4516:91078] +[CATra

In [15]:
#Conserved xylem marker XCP1
add_napari_layer(adata, column = 'TraesCS5D03G0377000') #
add_napari_layer(adata, column = 'TraesCS5A03G0388600') #



2024-06-09 16:44:40.575 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 16:44:40.707 python[4516:91078] +[CATransaction synchronize] called within transaction


In [35]:
# Conserved root cap marker
add_napari_layer(adata, column = 'TraesCS2D03G0347500') #



In [36]:
add_napari_layer(adata, column = 'TraesCS2D03G1303900') #
add_napari_layer(adata, column = 'TraesCS7D03G1072800LC') #
add_napari_layer(adata, column = 'TraesCS3A03G1088200') #
add_napari_layer(adata, column = 'TraesCS7D03G1077100') #
add_napari_layer(adata, column = 'TraesCS3B03G0004000') #
add_napari_layer(adata, column = 'TraesCS1A03G0045800') #
add_napari_layer(adata, column = 'TraesCS3D03G0000500') #
add_napari_layer(adata, column = 'TraesCS7A03G1125900') #
add_napari_layer(adata, column = 'TraesCS3B03G1278300') #
add_napari_layer(adata, column = 'TraesCS7D03G0949000') #
add_napari_layer(adata, column = 'TraesCS3B03G0134900') #
add_napari_layer(adata, column = 'TraesCS3A03G1099100') #
add_napari_layer(adata, column = 'TraesCS7D03G0969900LC') #
add_napari_layer(adata, column = 'TraesCS4A03G0828700') #
add_napari_layer(adata, column = 'TraesCS4B03G0721400') #
add_napari_layer(adata, column = 'TraesCS7A03G0992300LC') #
add_napari_layer(adata, column = 'TraesCS2B03G1534500') #
add_napari_layer(adata, column = 'TraesCS7A03G0992200LC') #
add_napari_layer(adata, column = 'TraesCS3A03G0103100') #
add_napari_layer(adata, column = 'TraesCS5D03G1173000') #
add_napari_layer(adata, column = 'TraesCS7B03G0829700LC') #
add_napari_layer(adata, column = 'TraesCS1D03G0362900') #
add_napari_layer(adata, column = 'TraesCS4A03G0065400') #
add_napari_layer(adata, column = 'TraesCS3D03G1015700') #
add_napari_layer(adata, column = 'TraesCS6B03G0242600') #
add_napari_layer(adata, column = 'TraesCS2B03G1540400') #
add_napari_layer(adata, column = 'TraesCS5B03G1313300') #
add_napari_layer(adata, column = 'TraesCS2B03G0426600LC') #
add_napari_layer(adata, column = 'TraesCS7B03G1286700') #
add_napari_layer(adata, column = 'TraesCS2A03G0309100LC') #
add_napari_layer(adata, column = 'TraesCS2D03G0324500') #
add_napari_layer(adata, column = 'TraesCS7A03G0992500LC') #
add_napari_layer(adata, column = 'TraesCS2D03G0324400') #
add_napari_layer(adata, column = 'TraesCS2A03G0309200') #
add_napari_layer(adata, column = 'TraesCS2B03G0426900') #
add_napari_layer(adata, column = 'TraesCS2B03G0426800') #


In [50]:
add_napari_layer(adata, column = 'TraesCS4A03G0828900') #
add_napari_layer(adata, column = 'TraesCS5A03G0269400') #
add_napari_layer(adata, column = 'TraesCS2B03G1537200') #
add_napari_layer(adata, column = 'TraesCS2D03G0800100') #
add_napari_layer(adata, column = 'TraesCS4B03G0964100') #
add_napari_layer(adata, column = 'TraesCS6A03G0972800') #
add_napari_layer(adata, column = 'TraesCS6B03G1189900') #
add_napari_layer(adata, column = 'TraesCS7B03G0593200') #
add_napari_layer(adata, column = 'TraesCS7A03G0779600') #
add_napari_layer(adata, column = 'TraesCS4D03G0621900') #
add_napari_layer(adata, column = 'TraesCS2A03G0733400') #
add_napari_layer(adata, column = 'TraesCS5D03G0300400') #
add_napari_layer(adata, column = 'TraesCS6B03G1074200') #
add_napari_layer(adata, column = 'TraesCS2B03G1540200') #
add_napari_layer(adata, column = 'TraesCS4A03G0828800') #
add_napari_layer(adata, column = 'TraesCS5B03G0297500') #
add_napari_layer(adata, column = 'TraesCSU03G0024000') #
add_napari_layer(adata, column = 'TraesCS2D03G1268900') #
add_napari_layer(adata, column = 'TraesCS7B03G0854100') #
add_napari_layer(adata, column = 'TraesCS6A03G0030300LC') #
add_napari_layer(adata, column = 'TraesCS2B03G0105800') #
add_napari_layer(adata, column = 'TraesCS5D03G1172900') #
add_napari_layer(adata, column = 'TraesCS3D03G0655100') #
add_napari_layer(adata, column = 'TraesCS3D03G1031300') #
add_napari_layer(adata, column = 'TraesCS6D03G0852900') #
add_napari_layer(adata, column = 'TraesCS6A03G0972100') #
add_napari_layer(adata, column = 'TraesCS6D03G0767200') #
add_napari_layer(adata, column = 'TraesCS5B03G1313200') #
add_napari_layer(adata, column = 'TraesCS3D03G0089800LC') #
add_napari_layer(adata, column = 'TraesCS7B03G0980100') #
add_napari_layer(adata, column = 'TraesCS2D03G0798800') #
add_napari_layer(adata, column = 'TraesCS7D03G0949100') #
add_napari_layer(adata, column = 'TraesCS3A03G0711700') #
add_napari_layer(adata, column = 'TraesCS1A03G0810500') #
add_napari_layer(adata, column = 'TraesCS1A03G0810300') #
add_napari_layer(adata, column = 'TraesCS7B03G1286600') #
add_napari_layer(adata, column = 'TraesCS2B03G0051900') #


In [38]:

#epidermis marker search
add_napari_layer(adata, column = 'TraesCS3B03G1515400') #
add_napari_layer(adata, column = 'TraesCS3D03G1192800') #
add_napari_layer(adata, column = 'TraesCS3B03G1008200') #
add_napari_layer(adata, column = 'TraesCS3B03G0439100') #
add_napari_layer(adata, column = 'TraesCS3A03G0365400') #








In [39]:
add_napari_layer(adata, column = 'TraesCS5D03G0906800') #
add_napari_layer(adata, column = 'TraesCS5B03G1000500') #
add_napari_layer(adata, column = 'TraesCS5A03G0951600') #
add_napari_layer(adata, column = 'TraesCS2D03G1242700') #
add_napari_layer(adata, column = 'TraesCS2A03G1281500') #
add_napari_layer(adata, column = 'TraesCS2B03G1468700') #
add_napari_layer(adata, column = 'TraesCS2B03G0914600') #
add_napari_layer(adata, column = 'TraesCS2D03G0769800') #


In [40]:
add_napari_layer(adata, column = 'TraesCS2D03G0321600') #
add_napari_layer(adata, column = 'TraesCS2A03G0303700') #
add_napari_layer(adata, column = 'TraesCS2B03G0423400') #
add_napari_layer(adata, column = 'TraesCS6B03G1184300') #
add_napari_layer(adata, column = 'TraesCS6D03G0848200') #
add_napari_layer(adata, column = 'TraesCS6B03G1184200') #
add_napari_layer(adata, column = 'TraesCS6A03G0967400') #
add_napari_layer(adata, column = 'TraesCS6A03G0967600') #
add_napari_layer(adata, column = 'TraesCS7B03G0299800') #
add_napari_layer(adata, column = 'TraesCS7D03G0468700') #
add_napari_layer(adata, column = 'TraesCS7A03G0484900') #


In [41]:
	
add_napari_layer(adata, column = 'TraesCS1B03G1166600') #
add_napari_layer(adata, column = 'TraesCS1D03G0951700') #



In [42]:
	
add_napari_layer(adata, column = 'TraesCS5D03G0654200') #
add_napari_layer(adata, column = 'TraesCS5B03G0722200') #
add_napari_layer(adata, column = 'TraesCS5A03G0684700') #


In [43]:


add_napari_layer(adata, column = 'TraesCS4B03G0721800') #
add_napari_layer(adata, column = 'TraesCS4A03G0065100') #


In [44]:

add_napari_layer(adata, column = 'TraesCS2A03G0858200') #


In [49]:

add_napari_layer(adata, column = 'TraesCS4D03G0217900') #


2024-06-12 16:34:09.248 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-12 16:34:09.408 python[4516:91078] +[CATransaction synchronize] called within transaction


In [46]:


add_napari_layer(adata, column = 'TraesCS3B03G1274600') #


In [47]:

add_napari_layer(adata, column = 'TraesCS3B03G1405100') #


In [48]:

add_napari_layer(adata, column = 'TraesCS2B03G1341500') #


In [16]:
#try endodermis

add_napari_layer(adata, column = 'TraesCS6B03G0815900') #
add_napari_layer(adata, column = 'TraesCS3D03G0402300') #
add_napari_layer(adata, column = 'TraesCS3A03G0779600') #
add_napari_layer(adata, column = 'TraesCS1B03G0155900') #
add_napari_layer(adata, column = 'TraesCS6B03G1068700') #
add_napari_layer(adata, column = 'TraesCS5D03G0917200') #
add_napari_layer(adata, column = 'TraesCS1A03G0789700') #
add_napari_layer(adata, column = 'TraesCS2B03G0825500') #
add_napari_layer(adata, column = 'TraesCS1B03G0906900') #
add_napari_layer(adata, column = 'TraesCS1A03G0117100') #
add_napari_layer(adata, column = 'TraesCS6D03G0894200') #
add_napari_layer(adata, column = 'TraesCS7B03G1305600') #
add_napari_layer(adata, column = 'TraesCS7A03G0413900') #
add_napari_layer(adata, column = 'TraesCS7D03G1292400') #
add_napari_layer(adata, column = 'TraesCS5A03G0526900') #
add_napari_layer(adata, column = 'TraesCS5B03G1013000') #
add_napari_layer(adata, column = 'TraesCS2A03G0303400') #
add_napari_layer(adata, column = 'TraesCS3B03G0508500') #
add_napari_layer(adata, column = 'TraesCS1D03G0995400') #
add_napari_layer(adata, column = 'TraesCS3D03G0402800') #
add_napari_layer(adata, column = 'TraesCS1B03G0907000') #
add_napari_layer(adata, column = 'TraesCS7A03G1365400') #
add_napari_layer(adata, column = 'TraesCS5B03G0538700') #
add_napari_layer(adata, column = 'TraesCS2D03G0321300') #
add_napari_layer(adata, column = 'TraesCS5D03G0492700') #
add_napari_layer(adata, column = 'TraesCS6A03G0661900') #
add_napari_layer(adata, column = 'TraesCS1D03G0105400') #
add_napari_layer(adata, column = 'TraesCS3D03G0720200') #
add_napari_layer(adata, column = 'TraesCS2B03G0423100') #
add_napari_layer(adata, column = 'TraesCS6A03G0889900') #
add_napari_layer(adata, column = 'TraesCS5A03G0962700') #
add_napari_layer(adata, column = 'TraesCS2D03G0692900') #
add_napari_layer(adata, column = 'TraesCS6D03G0764800') #
add_napari_layer(adata, column = 'TraesCS3B03G0508400') #
add_napari_layer(adata, column = 'TraesCS3B03G0898100') #
add_napari_layer(adata, column = 'TraesCS3D03G0402600') #
add_napari_layer(adata, column = 'TraesCS2A03G0749000') #
add_napari_layer(adata, column = 'TraesCS3A03G0429700') #
add_napari_layer(adata, column = 'TraesCS7D03G0401600') #
add_napari_layer(adata, column = 'TraesCS7B03G0218800') #


2024-06-12 23:14:22.763 python[1445:17999] +[CATransaction synchronize] called within transaction
2024-06-12 23:14:22.910 python[1445:17999] +[CATransaction synchronize] called within transaction


In [17]:
# Meristems
add_napari_layer(adata, column = 'TraesCS2B03G1047100') #
add_napari_layer(adata, column = 'TraesCS1B03G1035400') #
add_napari_layer(adata, column = 'TraesCS1A03G0723300') #
add_napari_layer(adata, column = 'TraesCS6A03G0197200') #
add_napari_layer(adata, column = 'TraesCS4B03G0567900') #


2024-06-12 23:40:20.278 python[1445:17999] +[CATransaction synchronize] called within transaction
2024-06-12 23:40:20.382 python[1445:17999] +[CATransaction synchronize] called within transaction


In [ ]:
# try epidermis cluster 1
add_napari_layer(adata, column = 'TraesCS6D03G0086400') #
add_napari_layer(adata, column = 'TraesCS6A03G0967400') #
add_napari_layer(adata, column = 'TraesCS4A03G1056600LC') #
add_napari_layer(adata, column = 'TraesCS5B03G0722200') #
add_napari_layer(adata, column = 'TraesCS6D03G0848200') #
add_napari_layer(adata, column = 'TraesCS6B03G1184300') #
add_napari_layer(adata, column = 'TraesCS4D03G0642700') #
add_napari_layer(adata, column = 'TraesCS2D03G0321600') #
add_napari_layer(adata, column = 'TraesCS4B03G0941900') #
add_napari_layer(adata, column = 'TraesCS4D03G0642600') #
add_napari_layer(adata, column = 'TraesCS6B03G1184200') #
add_napari_layer(adata, column = 'TraesCS3D03G1197600') #
add_napari_layer(adata, column = 'TraesCS4A03G0065100') #
add_napari_layer(adata, column = 'TraesCS4B03G0721600') #
add_napari_layer(adata, column = 'TraesCS4A03G0065200') #
add_napari_layer(adata, column = 'TraesCS2B03G0423400') #
add_napari_layer(adata, column = 'TraesCS6A03G0967600') #
add_napari_layer(adata, column = 'TraesCS1B03G0651500') #
add_napari_layer(adata, column = 'TraesCS3A03G0754700LC') #
add_napari_layer(adata, column = 'TraesCS3B03G0861400') #
add_napari_layer(adata, column = 'TraesCS4B03G0721800') #


In [26]:


add_napari_layer(adata, column = 'TraesCS1D03G0787400') #


In [22]:
#Monocot-specific  Conserved cortext marker across the 4 species
add_napari_layer(adata, column = 'TraesCS7B03G0675100') #

add_napari_layer(adata, column = 'TraesCS3A03G0974500') #



layer reused


2024-06-10 10:07:14.839 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-10 10:07:15.024 python[4516:91078] +[CATransaction synchronize] called within transaction


In [24]:
add_napari_layer(adata, column = 'TraesCS6B03G1184300') #


In [25]:
add_napari_layer(adata, column = 'TraesCS5A03G0363500') #


In [20]:
#Conserved epidermis marker
add_napari_layer(adata, column = 'TraesCS3A03G0635100') #




2024-06-09 21:43:01.642 python[4516:91078] +[CATransaction synchronize] called within transaction
2024-06-09 21:43:01.738 python[4516:91078] +[CATransaction synchronize] called within transaction


In [17]:
# unique to monocot clade

add_napari_layer(adata, column = 'TraesCS7B03G0675100') #


In [19]:
#Conserved Phloem marker


add_napari_layer(adata, column = 'TraesCS3B03G0001000') #



In [14]:


add_napari_layer(adata, column = 'TraesCS4D03G0434100') #


In [10]:
# For cluster 0
add_napari_layer(adata, column = 'TraesCS5B03G1052800') #
add_napari_layer(adata, column = 'TraesCS1B03G0069700') #
add_napari_layer(adata, column = 'TraesCS2B03G1501600') #
add_napari_layer(adata, column = 'TraesCS5A03G0948700') #
add_napari_layer(adata, column = 'TraesCS5A03G1006700') #


In [11]:
# For cluster 1

add_napari_layer(adata, column = 'TraesCS3A03G0754700LC') #
add_napari_layer(adata, column = 'TraesCS3B03G0861400') #
add_napari_layer(adata, column = 'TraesCS4B03G0721800') #
add_napari_layer(adata, column = 'TraesCS1B03G0651500') #
add_napari_layer(adata, column = 'TraesCS6A03G0967600') #





In [18]:
# For cluster 3 meristems

add_napari_layer(adata, column = 'TraesCS2B03G1047100') #
add_napari_layer(adata, column = 'TraesCS1B03G1035400') #
add_napari_layer(adata, column = 'TraesCS1A03G0723300') #
add_napari_layer(adata, column = 'TraesCS6A03G0197200') #
add_napari_layer(adata, column = 'TraesCS4B03G0567900') #








2024-05-28 14:52:37.023 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 14:52:37.173 python[91569:11853444] +[CATransaction synchronize] called within transaction


In [13]:
# For cluster 4 provascular cells


add_napari_layer(adata, column = 'TraesCS3A03G0016400') #
add_napari_layer(adata, column = 'TraesCS3D03G0013700') #
add_napari_layer(adata, column = 'TraesCS6A03G0661700LC') #
add_napari_layer(adata, column = 'TraesCS7A03G0475200') #
add_napari_layer(adata, column = 'TraesCS1B03G0051800') #

In [14]:
# For cluster 5 Unknown



add_napari_layer(adata, column = 'TraesCS3B03G0310800') #
add_napari_layer(adata, column = 'TraesCS3D03G0239000') #
add_napari_layer(adata, column = 'TraesCS4D03G0546300') #
add_napari_layer(adata, column = 'TraesCS7D03G0517200') #
add_napari_layer(adata, column = 'TraesCS3D03G0079700') #


In [20]:
#For cluster 6 Phloem


add_napari_layer(adata, column = 'TraesCS7A03G0757500') #
add_napari_layer(adata, column = 'TraesCS3D03G1070900') #
add_napari_layer(adata, column = 'TraesCS5B03G0138900') #
add_napari_layer(adata, column = 'TraesCS4B03G0262700') #
add_napari_layer(adata, column = 'TraesCS7D03G0728800') #


TraesCS6B03G0419800
TraesCS5B03G1121200
TraesCS4D03G0223700
TraesCS5D03G0150000LC
TraesCS5B03G0547100
TraesCS7B03G0588000
TraesCS3B03G0250100
TraesCS1D03G1000900
TraesCS5A03G0127300
TraesCS1B03G1236400
TraesCS4B03G0238600
TraesCS3A03G0715100
TraesCS1B03G0187300
TraesCS3B03G0820500
TraesCS3B03G1323800
TraesCS2A03G1202600
TraesCS3B03G1338300
TraesCS5D03G1199300
TraesCS3A03G1137900
TraesCS4A03G0535400
TraesCS4A03G0786200
TraesCS2D03G1158000
TraesCS4D03G0691800







In [21]:
#For cluster 6 Phloem
add_napari_layer(adata, column = 'TraesCS6B03G0419800') #
add_napari_layer(adata, column = 'TraesCS5B03G1121200') #
add_napari_layer(adata, column = 'TraesCS4D03G0223700') #
add_napari_layer(adata, column = 'TraesCS5D03G0150000LC') #
add_napari_layer(adata, column = 'TraesCS5B03G0547100') #
add_napari_layer(adata, column = 'TraesCS7B03G0588000') #
add_napari_layer(adata, column = 'TraesCS3B03G0250100') #
add_napari_layer(adata, column = 'TraesCS1D03G1000900') #
add_napari_layer(adata, column = 'TraesCS5A03G0127300') #
add_napari_layer(adata, column = 'TraesCS1B03G1236400') #
add_napari_layer(adata, column = 'TraesCS4B03G0238600') #
add_napari_layer(adata, column = 'TraesCS3A03G0715100') #
add_napari_layer(adata, column = 'TraesCS1B03G0187300') #
add_napari_layer(adata, column = 'TraesCS3B03G0820500') #
add_napari_layer(adata, column = 'TraesCS3B03G1323800') #
add_napari_layer(adata, column = 'TraesCS2A03G1202600') #
add_napari_layer(adata, column = 'TraesCS3B03G1338300') #
add_napari_layer(adata, column = 'TraesCS5D03G1199300') #
add_napari_layer(adata, column = 'TraesCS3A03G1137900') #
add_napari_layer(adata, column = 'TraesCS4A03G0535400') #
add_napari_layer(adata, column = 'TraesCS4A03G0786200') #
add_napari_layer(adata, column = 'TraesCS2D03G1158000') #
add_napari_layer(adata, column = 'TraesCS4D03G0691800') #


In [22]:

#For cluster 6 Phloem
add_napari_layer(adata, column = 'TraesCS3A03G1146800') #
add_napari_layer(adata, column = 'TraesCS1D03G0200100LC') #
add_napari_layer(adata, column = 'TraesCS6A03G0325300') #
add_napari_layer(adata, column = 'TraesCS3D03G0186500LC') #
add_napari_layer(adata, column = 'TraesCS4D03G0201600') #
add_napari_layer(adata, column = 'TraesCS7B03G0811900') #
add_napari_layer(adata, column = 'TraesCS5B03G1359600') #
add_napari_layer(adata, column = 'TraesCS5A03G0535600') #
add_napari_layer(adata, column = 'TraesCS1A03G0207300') #
add_napari_layer(adata, column = 'TraesCS7A03G1169600') #
add_napari_layer(adata, column = 'TraesCS2D03G0941600') #
add_napari_layer(adata, column = 'TraesCS7A03G0856200') #
add_napari_layer(adata, column = 'TraesCS3D03G1058700') #
add_napari_layer(adata, column = 'TraesCS4D03G0264900LC') #
add_napari_layer(adata, column = 'TraesCS2B03G1372800') #


In [17]:
#For cluster 11
add_napari_layer(adata, column = 'TraesCS5B03G1379500') #
add_napari_layer(adata, column = 'TraesCS5D03G1233700') #
add_napari_layer(adata, column = 'TraesCS3A03G0744700') #
add_napari_layer(adata, column = 'TraesCS2B03G0856800') #
add_napari_layer(adata, column = 'TraesCS2D03G0715400') #
add_napari_layer(adata, column = 'TraesCS2A03G0775400') #
add_napari_layer(adata, column = 'TraesCS7B03G0003800') #












/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/layers/shapes/_shapes_utils.py:709: RuntimeWarning: overflow encountered in square
  miter_lengths_squared = (miters**2).sum(axis=1)


In [19]:
# For cluster 12 Dividing cell

add_napari_layer(adata, column = 'TraesCS2D03G0803000') #
add_napari_layer(adata, column = 'TraesCS3B03G0839200LC') #
add_napari_layer(adata, column = 'TraesCS2A03G0863000') #
add_napari_layer(adata, column = 'TraesCS2B03G0955500') #
add_napari_layer(adata, column = 'TraesCS4D03G0742700') #
add_napari_layer(adata, column = 'TraesCS3A03G0796600') #










2024-05-28 14:59:50.528 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 14:59:50.644 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 15:01:38.431 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 15:01:38.559 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 15:01:46.739 python[91569:11853444] +[CATransaction synchronize] called within transaction
2024-05-28 15:01:48.355 python[91569:11853444] +[CATransaction synchronize] called within transaction


In [11]:
#For supplemental figure 11
add_napari_layer(adata, column = 'TraesCS5D03G0654200') #
add_napari_layer(adata, column = 'TraesCS5B03G0722200') #
add_napari_layer(adata, column = 'TraesCS5A03G0684700') #


In [11]:
#For supplemental figure 


add_napari_layer(adata, column = 'TraesCS1D03G0787400') #
add_napari_layer(adata, column = 'TraesCS3A03G0974500') #
add_napari_layer(adata, column = 'TraesCS3D03G0911000') #
add_napari_layer(adata, column = 'TraesCS3B03G1121400') #
add_napari_layer(adata, column = 'TraesCS3B03G1120300') #
add_napari_layer(adata, column = 'TraesCS2B03G1501600') #
add_napari_layer(adata, column = 'TraesCS2B03G1383800LC') #
add_napari_layer(adata, column = 'TraesCS1B03G0646400') #




2024-05-03 13:27:50.750 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:27:50.914 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:28:47.661 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:28:47.771 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:30:40.197 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:30:40.312 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:30:59.068 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 13:30:59.185 python[29934:3481336] +[CATransaction synchronize] called within transaction


In [33]:
add_napari_layer(adata, column = 'TraesCS1A03G0402200') #
add_napari_layer(adata, column = 'TraesCS1A03G0590400') #
add_napari_layer(adata, column = 'TraesCS1B03G0263100') #
add_napari_layer(adata, column = 'TraesCS1B03G0493300') #
add_napari_layer(adata, column = 'TraesCS1B03G0573100') #
add_napari_layer(adata, column = 'TraesCS1B03G0651800') #
add_napari_layer(adata, column = 'TraesCS1D03G0186800') #
add_napari_layer(adata, column = 'TraesCS1D03G0865900') #
add_napari_layer(adata, column = 'TraesCS2A03G0305200') #
add_napari_layer(adata, column = 'TraesCS2A03G0821300') #
add_napari_layer(adata, column = 'TraesCS2A03G0945900') #
add_napari_layer(adata, column = 'TraesCS2A03G0963600') #
add_napari_layer(adata, column = 'TraesCS2A03G0991200') #
add_napari_layer(adata, column = 'TraesCS2A03G1162200') #
add_napari_layer(adata, column = 'TraesCS2A03G1305300') #
add_napari_layer(adata, column = 'TraesCS2B03G0907700') #
add_napari_layer(adata, column = 'TraesCS2B03G0914600') #
add_napari_layer(adata, column = 'TraesCS2B03G1000500') #
add_napari_layer(adata, column = 'TraesCS2B03G1100500') #
add_napari_layer(adata, column = 'TraesCS2D03G0322900') #
add_napari_layer(adata, column = 'TraesCS2D03G0769800') #
add_napari_layer(adata, column = 'TraesCS2D03G0804100') #
add_napari_layer(adata, column = 'TraesCS2D03G0850700') #
add_napari_layer(adata, column = 'TraesCS2D03G1175000') #
add_napari_layer(adata, column = 'TraesCS2D03G1190900') #
add_napari_layer(adata, column = 'TraesCS3A03G0017400') #
add_napari_layer(adata, column = 'TraesCS3B03G0018300') #
add_napari_layer(adata, column = 'TraesCS3B03G0391800') #
add_napari_layer(adata, column = 'TraesCS3B03G0731700') #
add_napari_layer(adata, column = 'TraesCS3B03G0885500') #
add_napari_layer(adata, column = 'TraesCS3B03G1344800') #
add_napari_layer(adata, column = 'TraesCS3B03G1461300') #
add_napari_layer(adata, column = 'TraesCS3D03G0332500') #
add_napari_layer(adata, column = 'TraesCS3D03G0712500') #
add_napari_layer(adata, column = 'TraesCS4A03G0280400') #
add_napari_layer(adata, column = 'TraesCS4B03G0112400') #
add_napari_layer(adata, column = 'TraesCS4D03G0095600') #
add_napari_layer(adata, column = 'TraesCS4D03G0434100') #
add_napari_layer(adata, column = 'TraesCS4D03G0679400') #
add_napari_layer(adata, column = 'TraesCS5A03G0417300') #
add_napari_layer(adata, column = 'TraesCS5A03G0951600') #
add_napari_layer(adata, column = 'TraesCS5A03G1035600') #
add_napari_layer(adata, column = 'TraesCS5A03G1145500') #
add_napari_layer(adata, column = 'TraesCS5D03G0383900') #
add_napari_layer(adata, column = 'TraesCS5D03G1079100') #
add_napari_layer(adata, column = 'TraesCS6A03G0353200') #
add_napari_layer(adata, column = 'TraesCS6D03G0765000') #
add_napari_layer(adata, column = 'TraesCS7A03G0045400') #
add_napari_layer(adata, column = 'TraesCS7A03G0446200') #
add_napari_layer(adata, column = 'TraesCS7A03G0475600') #
add_napari_layer(adata, column = 'TraesCS7D03G0992000') #




In [20]:
# For Rahul
add_napari_layer(adata, column = 'TraesCS1D03G0941600') # 
add_napari_layer(adata, column = 'TraesCS2D03G0850400') # 
add_napari_layer(adata, column = 'TraesCS5A03G1145500') # 
add_napari_layer(adata, column = 'TraesCS5D03G1079100') # 
add_napari_layer(adata, column = 'TraesCS2B03G1000500') # 
add_napari_layer(adata, column = 'TraesCS3B03G0903600') # 
add_napari_layer(adata, column = 'TraesCS7A03G0475600') # 
add_napari_layer(adata, column = 'TraesCS2D03G0769800') # 
add_napari_layer(adata, column = 'TraesCS3D03G0712500') # 








layer reused
layer reused
layer reused
layer reused


In [21]:
add_napari_layer(adata, column='TraesCS4B03G0883700')

In [16]:
# Epidermis Top 2
add_napari_layer(adata, column = 'TraesCS2D03G0324500') # 
add_napari_layer(adata, column = 'TraesCS2D03G0324400') # 



layer reused
layer reused


In [10]:
# Phloem Top 20
add_napari_layer(adata, column = 'TraesCS2B03G0108200') #
add_napari_layer(adata, column = 'TraesCS2D03G0068900') #
add_napari_layer(adata, column = 'TraesCS3B03G0900500') #
add_napari_layer(adata, column = 'TraesCS2A03G0073400') #
add_napari_layer(adata, column = 'TraesCS3D03G0721900') #
add_napari_layer(adata, column = 'TraesCS3A03G0782300') #
add_napari_layer(adata, column = 'TraesCS1D03G0884600') #
add_napari_layer(adata, column = 'TraesCS3D03G0599200') #
add_napari_layer(adata, column = 'TraesCS6A03G0666600') #
add_napari_layer(adata, column = 'TraesCS3A03G0654700') #
add_napari_layer(adata, column = 'TraesCS1D03G0605000') #
add_napari_layer(adata, column = 'TraesCS1B03G0733300') #
add_napari_layer(adata, column = 'TraesCS1A03G0647000') #
add_napari_layer(adata, column = 'TraesCS1A03G0906900') #
add_napari_layer(adata, column = 'TraesCS3B03G0742300') #
add_napari_layer(adata, column = 'TraesCS6A03G0202900') #
add_napari_layer(adata, column = 'TraesCS4D03G0020800') #
add_napari_layer(adata, column = 'TraesCS7D03G0500500') #
add_napari_layer(adata, column = 'TraesCS5D03G0557600') #
add_napari_layer(adata, column = 'TraesCS3B03G0486000') #

2024-05-03 12:44:13.876 python[29934:3481336] +[CATransaction synchronize] called within transaction
2024-05-03 12:44:14.439 python[29934:3481336] +[CATransaction synchronize] called within transaction


In [13]:
# Phloem Top 40
add_napari_layer(adata, column = 'TraesCS6B03G0810000') #
add_napari_layer(adata, column = 'TraesCS5A03G0536500') #
add_napari_layer(adata, column = 'TraesCS6B03G0398800') #
add_napari_layer(adata, column = 'TraesCS6A03G0306800') #
add_napari_layer(adata, column = 'TraesCS4D03G0760800') #
add_napari_layer(adata, column = 'TraesCS7B03G0335800') #
add_napari_layer(adata, column = 'TraesCS4B03G0030200') #
add_napari_layer(adata, column = 'TraesCS6B03G0270900') #
add_napari_layer(adata, column = 'TraesCS3A03G1157200') #
add_napari_layer(adata, column = 'TraesCS5A03G0819000') #
add_napari_layer(adata, column = 'TraesCS3B03G1380200') #
add_napari_layer(adata, column = 'TraesCS4A03G0754700') #
add_napari_layer(adata, column = 'TraesCS3B03G0494600') #
add_napari_layer(adata, column = 'TraesCS4B03G0725300') #
add_napari_layer(adata, column = 'TraesCS3A03G0407400') #
add_napari_layer(adata, column = 'TraesCS6B03G0419400') #
add_napari_layer(adata, column = 'TraesCS1D03G0349000') #
add_napari_layer(adata, column = 'TraesCS3A03G0709800') #
add_napari_layer(adata, column = 'TraesCS3D03G0382200') #
add_napari_layer(adata, column = 'TraesCS4D03G0091800') #



In [14]:
# Phloem  Top 50
add_napari_layer(adata, column = 'TraesCS3D03G1105300') #
add_napari_layer(adata, column = 'TraesCS1A03G0542800') #
add_napari_layer(adata, column = 'TraesCS3A03G0390900') #
add_napari_layer(adata, column = 'TraesCS1B03G0715800LC') #
add_napari_layer(adata, column = 'TraesCS1D03G0587800') #
add_napari_layer(adata, column = 'TraesCS4A03G0687000') #
add_napari_layer(adata, column = 'TraesCS6D03G0279900') #
add_napari_layer(adata, column = 'TraesCS1A03G0361700') #
add_napari_layer(adata, column = 'TraesCS1B03G0450900') #
add_napari_layer(adata, column = 'TraesCS4B03G0108300') #
add_napari_layer(adata, column = 'TraesCS1A03G0626900') #
add_napari_layer(adata, column = 'TraesCS5B03G0548000') #
add_napari_layer(adata, column = 'TraesCS5A03G1176600LC') #
add_napari_layer(adata, column = 'TraesCS7B03G0047600') #
add_napari_layer(adata, column = 'TraesCS2D03G0137900') #
add_napari_layer(adata, column = 'TraesCS4A03G0321300') #
add_napari_layer(adata, column = 'TraesCS5D03G0143700') #
add_napari_layer(adata, column = 'TraesCS2A03G0138900') #





/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in

## Add gene laryers

In [40]:
# cluster0  epidermis
add_napari_layer(adata, column = 'TraesCS7B03G0854100')  # uncharacterized protein LOC119341924
add_napari_layer(adata, column = 'TraesCS5D03G1173000')  # glycine-rich cell wall structural protein 2-like
add_napari_layer(adata, column = 'TraesCS7D03G1077100')  # agglutinin isolectin 3-like
add_napari_layer(adata, column = 'TraesCS6B03G0004100')  # N/A
add_napari_layer(adata, column = 'TraesCS5D03G1172900')  # putative glycine-rich cell wall structural protein 1
add_napari_layer(adata, column = 'TraesCS5B03G0722200')  # auxin-induced in root cultures protein 12-like
add_napari_layer(adata, column = 'TraesCS4A03G1056600LC') # N/A
add_napari_layer(adata, column = 'TraesCS3D03G0089800LC') # uncharacterized protein LOC119355594
add_napari_layer(adata, column = 'TraesCS5D03G0654200')  # N/A
add_napari_layer(adata, column = 'TraesCS2D03G1268900')  # auxin-induced in root cultures protein 12-like
add_napari_layer(adata, column = 'TraesCS3A03G0754700LC') # 4-hydroxy-7-methoxy-3-oxo-3,4-dihydro-2H-1,4-benzoxazin-2-yl glucoside beta-D-glucosidase 1b, chloroplastic
add_napari_layer(adata, column = 'TraesCS6A03G0967600')  # N/A
add_napari_layer(adata, column = 'TraesCS2A03G0038300')  # germin-like protein 1-1
add_napari_layer(adata, column = 'TraesCS2D03G0039600')  # probable apyrase 3
add_napari_layer(adata, column = 'TraesCS2A03G0858200')  
add_napari_layer(adata, column = 'TraesCS3B03G1405100')  
add_napari_layer(adata, column = 'TraesCS7D03G0620600')  
add_napari_layer(adata, column = 'TraesCS2D03G0324500')  
add_napari_layer(adata, column = 'TraesCS1B03G1040000')  
add_napari_layer(adata, column = 'TraesCS7B03G0854100')  












layer reused


In [41]:
# cluster 1  provascular cells
add_napari_layer(adata, column = 'TraesCS3A03G0016400')  # WALLS ARE THIN 1-like
add_napari_layer(adata, column = 'TraesCS1B03G0051800')  # Bowman-Birk type trypsin inhibitor-like
add_napari_layer(adata, column = 'TraesCS3B03G0036500')  # WALLS ARE THIN 1-like
add_napari_layer(adata, column = 'TraesCS3D03G0013700')  # WALLS ARE THIN 1-like
add_napari_layer(adata, column = 'TraesCS1B03G0053600')  # Bowman-Birk type trypsin inhibitor-like
add_napari_layer(adata, column = 'TraesCS5B03G0678400LC')  # N/A
add_napari_layer(adata, column = 'TraesCS2A03G0412800') # mavicyanin-like
add_napari_layer(adata, column = 'TraesCS3B03G0017200') # WALLS ARE THIN 1-like
add_napari_layer(adata, column = 'TraesCS2B03G0107600')  # uncharacterized protein C594.04c-like
add_napari_layer(adata, column = 'TraesCS6A03G0661700LC')  # N/A





/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [42]:
# cluster 2  S phase 
add_napari_layer(adata, column = 'TraesCS1A03G0879600')  
add_napari_layer(adata, column = 'TraesCS1A03G0879000')  
add_napari_layer(adata, column = 'TraesCS1D03G0853100')  
add_napari_layer(adata, column = 'TraesCS3A03G0316900') 
add_napari_layer(adata, column = 'TraesCS1A03G0723300') 
add_napari_layer(adata, column = 'TraesCS2B03G1047100')  
add_napari_layer(adata, column = 'TraesCS1A03G0090800') 
add_napari_layer(adata, column = 'TraesCS1B03G1050300') 
add_napari_layer(adata, column = 'TraesCS6A03G0076200')  
add_napari_layer(adata, column = 'TraesCS1B03G1035400') 











In [43]:
# cluster 3 Cortex
add_napari_layer(adata, column = 'TraesCS1D03G0787300')  
add_napari_layer(adata, column = 'TraesCS1D03G0787400')  
add_napari_layer(adata, column = 'TraesCS2D03G0925700')  
add_napari_layer(adata, column = 'TraesCS1A03G0817100') 
add_napari_layer(adata, column = 'TraesCS4B03G0814800') 
add_napari_layer(adata, column = 'TraesCS4B03G0814900') 
add_napari_layer(adata, column = 'TraesCS3A03G0974500') 
add_napari_layer(adata, column = 'TraesCS1B03G0768600') 
add_napari_layer(adata, column = 'TraesCS1B03G0646400') 
add_napari_layer(adata, column = 'TraesCS1A03G0554700') 








In [44]:
# cluster 3 Cortex 
add_napari_layer(adata, column = 'TraesCS1D03G0534700')  
add_napari_layer(adata, column = 'TraesCS1A03G0554500')  
add_napari_layer(adata, column = 'TraesCS3D03G0911000')  
add_napari_layer(adata, column = 'TraesCS1D03G0534300')  
add_napari_layer(adata, column = 'TraesCS1A03G0676300')  
add_napari_layer(adata, column = 'TraesCS1B03G0645400')  
add_napari_layer(adata, column = 'TraesCS1B03G0645800')  
add_napari_layer(adata, column = 'TraesCS6B03G1181300')  
add_napari_layer(adata, column = 'TraesCS7B03G0820900')  
add_napari_layer(adata, column = 'TraesCS4B03G0809400')  
add_napari_layer(adata, column = 'TraesCS2B03G1383800LC')  
add_napari_layer(adata, column = 'TraesCS3D03G0146400')  
add_napari_layer(adata, column = 'TraesCS4B03G0094500')  
add_napari_layer(adata, column = 'TraesCS7B03G0675100')  
add_napari_layer(adata, column = 'TraesCS2B03G1501600')  
add_napari_layer(adata, column = 'TraesCS4A03G1016800')  
add_napari_layer(adata, column = 'TraesCS7A03G0143900')  
add_napari_layer(adata, column = 'TraesCSU03G0510000LC')  
add_napari_layer(adata, column = 'TraesCS7A03G0983300')  
add_napari_layer(adata, column = 'TraesCS1A03G0676400')  













In [45]:
# cluster 4 unknown
add_napari_layer(adata, column = 'TraesCS1B03G0069700')  
add_napari_layer(adata, column = 'TraesCS1A03G0098300')  
add_napari_layer(adata, column = 'TraesCS2D03G0859100LC')  
add_napari_layer(adata, column = 'TraesCS3B03G0087500')  
add_napari_layer(adata, column = 'TraesCS1B03G0068200')  
add_napari_layer(adata, column = 'TraesCS4B03G0489300')  
add_napari_layer(adata, column = 'TraesCS1B03G0048500')  
add_napari_layer(adata, column = 'TraesCS2D03G0798400')  
add_napari_layer(adata, column = 'TraesCS5B03G1052800')  
add_napari_layer(adata, column = 'TraesCS4D03G0435400')  






In [46]:
# cluster 5 unknown
add_napari_layer(adata, column = 'TraesCS7D03G0370000')  
add_napari_layer(adata, column = 'TraesCS3A03G1063100')  
add_napari_layer(adata, column = 'TraesCS7A03G0384500')  
add_napari_layer(adata, column = 'TraesCS3D03G0991100')  
add_napari_layer(adata, column = 'TraesCS3A03G0209900')  
add_napari_layer(adata, column = 'TraesCS7B03G0170600')  
add_napari_layer(adata, column = 'TraesCS6A03G0822100')  
add_napari_layer(adata, column = 'TraesCS3B03G1230400')  
add_napari_layer(adata, column = 'TraesCS3D03G0946200')  
add_napari_layer(adata, column = 'TraesCS7A03G0214300')  







In [47]:
# cluster 6 
add_napari_layer(adata, column = 'TraesCSU03G0016400')  
add_napari_layer(adata, column = 'TraesCS4B03G0178500')  
add_napari_layer(adata, column = 'TraesCS4A03G1041600')  
add_napari_layer(adata, column = 'TraesCS1D03G0527600')  
add_napari_layer(adata, column = 'TraesCS4A03G1041500')  
add_napari_layer(adata, column = 'TraesCS2D03G0468100')  
add_napari_layer(adata, column = 'TraesCS1A03G0547600')  
add_napari_layer(adata, column = 'TraesCS2A03G0462100')  
add_napari_layer(adata, column = 'TraesCS4D03G0146400')  
add_napari_layer(adata, column = 'TraesCS2B03G0582100')  






In [48]:
# cluster 6 
add_napari_layer(adata, column = 'TraesCS7B03G0410500')  
add_napari_layer(adata, column = 'TraesCS4D03G0146500')  
add_napari_layer(adata, column = 'TraesCS2B03G0611400')  
add_napari_layer(adata, column = 'TraesCS7A03G0816500')  
add_napari_layer(adata, column = 'TraesCS2A03G0508300')  
add_napari_layer(adata, column = 'TraesCS7A03G0586100')  
add_napari_layer(adata, column = 'TraesCS2D03G0496100')  
add_napari_layer(adata, column = 'TraesCS2B03G1319000')  
add_napari_layer(adata, column = 'TraesCS4B03G0989000')  
add_napari_layer(adata, column = 'TraesCS5D03G0906800')  













In [49]:
# cluster 7  likely procambium
add_napari_layer(adata, column = 'TraesCS7A03G0757500') 
add_napari_layer(adata, column = 'TraesCS7D03G0728800') 
add_napari_layer(adata, column = 'TraesCS5B03G1121200') 
add_napari_layer(adata, column = 'TraesCS3D03G1070900') 
add_napari_layer(adata, column = 'TraesCS4B03G0262700') 
add_napari_layer(adata, column = 'TraesCS6B03G0419800') 
add_napari_layer(adata, column = 'TraesCS5B03G0138900') 
add_napari_layer(adata, column = 'TraesCS5B03G0547100') 
add_napari_layer(adata, column = 'TraesCS7B03G0588000') 
add_napari_layer(adata, column = 'TraesCS4D03G0223700') 











In [50]:
# cluster 7   likely procambium?
add_napari_layer(adata, column = 'TraesCS5D03G0150000LC') 
add_napari_layer(adata, column = 'TraesCS1D03G1000900') 
add_napari_layer(adata, column = 'TraesCS4B03G0238600') 
add_napari_layer(adata, column = 'TraesCS3A03G0715100') 
add_napari_layer(adata, column = 'TraesCS5A03G0127300') 
add_napari_layer(adata, column = 'TraesCS1B03G0187300') 
add_napari_layer(adata, column = 'TraesCS3B03G0820500') 
add_napari_layer(adata, column = 'TraesCS3B03G0250100') 
add_napari_layer(adata, column = 'TraesCS4D03G0201600') 
add_napari_layer(adata, column = 'TraesCS5A03G0535600') 








In [51]:
# cluster 8  unknown
add_napari_layer(adata, column = 'TraesCS3B03G0898100') 
add_napari_layer(adata, column = 'TraesCS3D03G0402600') 
add_napari_layer(adata, column = 'TraesCS7D03G0401600') 
add_napari_layer(adata, column = 'TraesCS3A03G0429700') 
add_napari_layer(adata, column = 'TraesCS3B03G0508400') 
add_napari_layer(adata, column = 'TraesCS2A03G0749000') 
add_napari_layer(adata, column = 'TraesCS7B03G0218800') 
add_napari_layer(adata, column = 'TraesCS2B03G0423100') 
add_napari_layer(adata, column = 'TraesCS2D03G0321300') 
add_napari_layer(adata, column = 'TraesCS2D03G0692900') 












In [52]:
# cluster 8  unknown
add_napari_layer(adata, column = 'TraesCS1D03G0105400') 
add_napari_layer(adata, column = 'TraesCS6A03G0661900') 
add_napari_layer(adata, column = 'TraesCS6D03G0894200') 
add_napari_layer(adata, column = 'TraesCS3D03G1192600') 
add_napari_layer(adata, column = 'TraesCS7A03G0413900') 
add_napari_layer(adata, column = 'TraesCS1A03G0117100') 
add_napari_layer(adata, column = 'TraesCS6D03G0670000') 
add_napari_layer(adata, column = 'TraesCS1B03G0155900') 
add_napari_layer(adata, column = 'TraesCS2B03G1538800') 
add_napari_layer(adata, column = 'TraesCS3D03G0686600') 












In [53]:
# cluster 9  unknown  no obvious pattern in root sections
add_napari_layer(adata, column = 'TraesCS3D03G0079600') 
add_napari_layer(adata, column = 'TraesCS3D03G0080400') 
add_napari_layer(adata, column = 'TraesCS4D03G0298500') 
add_napari_layer(adata, column = 'TraesCS3D03G0239000') 
add_napari_layer(adata, column = 'TraesCS3A03G0070700') 
add_napari_layer(adata, column = 'TraesCS3B03G0312200') 
add_napari_layer(adata, column = 'TraesCS6D03G0752200') 
add_napari_layer(adata, column = 'TraesCS6B03G1056600') 
add_napari_layer(adata, column = 'TraesCS3D03G0238500') 
add_napari_layer(adata, column = 'TraesCS3B03G0310600') 













In [55]:
# cluster 10  xylem
add_napari_layer(adata, column = 'TraesCS5A03G0388600') 
add_napari_layer(adata, column = 'TraesCS5B03G0387000') 
add_napari_layer(adata, column = 'TraesCS5D03G0377000') 
add_napari_layer(adata, column = 'TraesCS1D03G0049400') 
add_napari_layer(adata, column = 'TraesCS1A03G0062800') 
add_napari_layer(adata, column = 'TraesCS2B03G0484900') 
add_napari_layer(adata, column = 'TraesCS6D03G0082600') 
add_napari_layer(adata, column = 'TraesCS2A03G0730800') 
add_napari_layer(adata, column = 'TraesCS7B03G0805100') 
add_napari_layer(adata, column = 'TraesCS3B03G1344800') 
add_napari_layer(adata, column = 'TraesCSU03G0020100') 
add_napari_layer(adata, column = 'TraesCS2B03G0806600') 
add_napari_layer(adata, column = 'TraesCS6D03G0654300') 
add_napari_layer(adata, column = 'TraesCS2D03G0673900') 
add_napari_layer(adata, column = 'TraesCS6A03G0080100') 
add_napari_layer(adata, column = 'TraesCS3B03G0012900') 
add_napari_layer(adata, column = 'TraesCS7A03G0969300') 
add_napari_layer(adata, column = 'TraesCS3D03G0008300') 
add_napari_layer(adata, column = 'TraesCS7A03G0446200') 
add_napari_layer(adata, column = 'TraesCS1B03G0066500') 

























In [56]:
# cluster 11  epidermis + cortex
add_napari_layer(adata, column = 'TraesCS5A03G0907300') 
add_napari_layer(adata, column = 'TraesCS1D03G0244200') 
add_napari_layer(adata, column = 'TraesCS6D03G0846300') 
add_napari_layer(adata, column = 'TraesCS2B03G1536600') 
add_napari_layer(adata, column = 'TraesCS2B03G1500200')  # epidermis + cortex
add_napari_layer(adata, column = 'TraesCS7A03G1099200') 
add_napari_layer(adata, column = 'TraesCS5B03G0958400') 
add_napari_layer(adata, column = 'TraesCSU03G0020900') 
add_napari_layer(adata, column = 'TraesCS7B03G0820900') 
add_napari_layer(adata, column = 'TraesCS7A03G0983300') 










In [57]:
# cluster 12 
add_napari_layer(adata, column = 'TraesCS2D03G0803000') 
add_napari_layer(adata, column = 'TraesCS2A03G0863000') 
add_napari_layer(adata, column = 'TraesCS3B03G0839200LC') 
add_napari_layer(adata, column = 'TraesCS2B03G0955500') 
add_napari_layer(adata, column = 'TraesCS4D03G0742700') 
add_napari_layer(adata, column = 'TraesCS3A03G0935100') 
add_napari_layer(adata, column = 'TraesCS1B03G0616100') 
add_napari_layer(adata, column = 'TraesCS4B03G0833500') 
add_napari_layer(adata, column = 'TraesCS3B03G0918400') 
add_napari_layer(adata, column = 'TraesCS3B03G1202600') 
add_napari_layer(adata, column = 'TraesCS3D03G0735900') 
add_napari_layer(adata, column = 'TraesCS2D03G0129500') 
add_napari_layer(adata, column = 'TraesCS1A03G0526900') 
add_napari_layer(adata, column = 'TraesCS1D03G0508400') 
add_napari_layer(adata, column = 'TraesCS1D03G0735800') 
add_napari_layer(adata, column = 'TraesCS1B03G0879500') 
add_napari_layer(adata, column = 'TraesCS2A03G0132400') 
add_napari_layer(adata, column = 'TraesCS3D03G0969300') 
add_napari_layer(adata, column = 'TraesCS7D03G1262100') 
add_napari_layer(adata, column = 'TraesCS3A03G0796600') 





















In [13]:
# res0.24  cluster 0 initials
add_napari_layer(adata, column = 'TraesCS5B03G1052800') 
add_napari_layer(adata, column = 'TraesCS1B03G0069700') 
add_napari_layer(adata, column = 'TraesCS2B03G1501600') 
add_napari_layer(adata, column = 'TraesCS5A03G0948700') 
add_napari_layer(adata, column = 'TraesCS5A03G1006700') 
add_napari_layer(adata, column = 'TraesCS3B03G0498100') 
add_napari_layer(adata, column = 'TraesCS4B03G0489300') 
add_napari_layer(adata, column = 'TraesCS3D03G0146400') 
add_napari_layer(adata, column = 'TraesCS1B03G0841700') 
add_napari_layer(adata, column = 'TraesCS3D03G0061500') 
add_napari_layer(adata, column = 'TraesCS1B03G0768600') 
add_napari_layer(adata, column = 'TraesCS2A03G0313900') 
add_napari_layer(adata, column = 'TraesCS1D03G0703900') 
add_napari_layer(adata, column = 'TraesCS4B03G0939900') 
add_napari_layer(adata, column = 'TraesCS3A03G0812200') 
add_napari_layer(adata, column = 'TraesCS3B03G0087700') 
add_napari_layer(adata, column = 'TraesCS4D03G0668900') 
add_napari_layer(adata, column = 'TraesCS2D03G0859100LC') 
add_napari_layer(adata, column = 'TraesCS2B03G1500200') 
add_napari_layer(adata, column = 'TraesCS3D03G0392300') 












In [18]:
# best_annotation_endodermis ?
add_napari_layer(adata, column = 'TraesCS7D03G1077100') 
add_napari_layer(adata, column = 'TraesCS7B03G0854100') 
add_napari_layer(adata, column = 'TraesCS3D03G0900300') 
add_napari_layer(adata, column = 'TraesCS3A03G0287200') 
add_napari_layer(adata, column = 'TraesCS6B03G0004100') 
add_napari_layer(adata, column = 'TraesCS6D03G0848200') 
add_napari_layer(adata, column = 'TraesCS6A03G0967600') 


  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/asyncio/base_events.py", line 570, in run_forever

In [14]:
# cluster 1   ??? epidermis ???
add_napari_layer(adata, column = 'TraesCS4A03G0065200') 
add_napari_layer(adata, column = 'TraesCS4D03G0642700') 
add_napari_layer(adata, column = 'TraesCS4D03G0642600') 
add_napari_layer(adata, column = 'TraesCS4A03G0065100') 
add_napari_layer(adata, column = 'TraesCS3D03G1197600') 
add_napari_layer(adata, column = 'TraesCS4B03G0721600') 
add_napari_layer(adata, column = 'TraesCS4B03G0721800') 
add_napari_layer(adata, column = 'TraesCS6B03G0123000') 
add_napari_layer(adata, column = 'TraesCS2D03G0321600') 
add_napari_layer(adata, column = 'TraesCS3A03G0287200') 
add_napari_layer(adata, column = 'TraesCS6D03G0086400') 
add_napari_layer(adata, column = 'TraesCS3D03G0900300') 
add_napari_layer(adata, column = 'TraesCS2B03G0423400') 
add_napari_layer(adata, column = 'TraesCS3B03G0861400') 
add_napari_layer(adata, column = 'TraesCS7B03G0299800') 
add_napari_layer(adata, column = 'TraesCS6A03G0967600') 
add_napari_layer(adata, column = 'TraesCS4B03G0941900') 
add_napari_layer(adata, column = 'TraesCS4A03G1056600LC') 
add_napari_layer(adata, column = 'TraesCS1B03G0651500') 
add_napari_layer(adata, column = 'TraesCS3A03G0754700LC') 
add_napari_layer(adata, column = 'TraesCS6B03G1184300') 
add_napari_layer(adata, column = 'TraesCS5B03G0722200') 
add_napari_layer(adata, column = 'TraesCS6B03G1184200') 
add_napari_layer(adata, column = 'TraesCS6A03G0967400') 
add_napari_layer(adata, column = 'TraesCS6D03G0848200') 












In [16]:
# cluster 2  Cortex 
add_napari_layer(adata, column = 'TraesCS2B03G1096500') 
add_napari_layer(adata, column = 'TraesCS7B03G1036600') 
add_napari_layer(adata, column = 'TraesCS2D03G0925700') 
add_napari_layer(adata, column = 'TraesCS3B03G1121400') 
add_napari_layer(adata, column = 'TraesCS6B03G0811700') 
add_napari_layer(adata, column = 'TraesCS2A03G0988000') 
add_napari_layer(adata, column = 'TraesCS7D03G1271400') 
add_napari_layer(adata, column = 'TraesCS3A03G0974500') 
add_napari_layer(adata, column = 'TraesCS4D03G0728500') 
add_napari_layer(adata, column = 'TraesCS1A03G0817100') 
add_napari_layer(adata, column = 'TraesCS7A03G1345700') 
add_napari_layer(adata, column = 'TraesCS1A03G0026200') 
add_napari_layer(adata, column = 'TraesCS1D03G0534600') 
add_napari_layer(adata, column = 'TraesCS6D03G0553800') 
add_napari_layer(adata, column = 'TraesCS1D03G0787400') 
add_napari_layer(adata, column = 'TraesCS1D03G0787300') 
add_napari_layer(adata, column = 'TraesCS1B03G1174700') 
add_napari_layer(adata, column = 'TraesCS1B03G1234700') 
add_napari_layer(adata, column = 'TraesCS1A03G0554700') 
add_napari_layer(adata, column = 'TraesCS4B03G0814800') 
add_napari_layer(adata, column = 'TraesCS3D03G0911000') 
add_napari_layer(adata, column = 'TraesCS3B03G1120300') 
add_napari_layer(adata, column = 'TraesCS4B03G0094500') 
add_napari_layer(adata, column = 'TraesCS1D03G0534700') 
add_napari_layer(adata, column = 'TraesCS1B03G0645400') 
add_napari_layer(adata, column = 'TraesCS1B03G0646400') 
add_napari_layer(adata, column = 'TraesCS7A03G0983300') 
add_napari_layer(adata, column = 'TraesCSU03G0510000LC') 
add_napari_layer(adata, column = 'TraesCS7A03G0143900') 
add_napari_layer(adata, column = 'TraesCS7B03G0820900') 
add_napari_layer(adata, column = 'TraesCS6B03G1181300') 
add_napari_layer(adata, column = 'TraesCS1B03G0645800') 
add_napari_layer(adata, column = 'TraesCS4B03G0814900') 
add_napari_layer(adata, column = 'TraesCS1A03G0554500') 
add_napari_layer(adata, column = 'TraesCS1B03G0768600') 
add_napari_layer(adata, column = 'TraesCS1A03G0676300') 
add_napari_layer(adata, column = 'TraesCS2B03G1501600') 
add_napari_layer(adata, column = 'TraesCS2B03G1383800LC') 
add_napari_layer(adata, column = 'TraesCS1D03G0534300') 
add_napari_layer(adata, column = 'TraesCS3D03G0146400') 

















In [17]:
# cluster 4  provascular cells
add_napari_layer(adata, column = 'TraesCS3B03G0036500') 
add_napari_layer(adata, column = 'TraesCS4D03G0562400') 
add_napari_layer(adata, column = 'TraesCS1A03G0532100') 
add_napari_layer(adata, column = 'TraesCS1B03G0622300') 
add_napari_layer(adata, column = 'TraesCS7A03G0475200') 
add_napari_layer(adata, column = 'TraesCS6D03G0573300') 
add_napari_layer(adata, column = 'TraesCS2B03G0107600') 
add_napari_layer(adata, column = 'TraesCS3D03G0013700') 
add_napari_layer(adata, column = 'TraesCS5D03G0346300') 
add_napari_layer(adata, column = 'TraesCS3A03G0016400') 
add_napari_layer(adata, column = 'TraesCS1B03G0050600') 
add_napari_layer(adata, column = 'TraesCS6D03G0550100LC') 
add_napari_layer(adata, column = 'TraesCS6A03G0661700LC') 
add_napari_layer(adata, column = 'TraesCS3D03G0132200') 
add_napari_layer(adata, column = 'TraesCS2A03G0412800') 
add_napari_layer(adata, column = 'TraesCS1B03G0053600') 
add_napari_layer(adata, column = 'TraesCS1B03G0051800') 
add_napari_layer(adata, column = 'TraesCS7D03G1240300') 
add_napari_layer(adata, column = 'TraesCS5B03G0678400LC') 
add_napari_layer(adata, column = 'TraesCS2B03G1307900') 























In [18]:
# cluster 5 
add_napari_layer(adata, column = 'TraesCS3B03G0310800') 
add_napari_layer(adata, column = 'TraesCS3D03G0239000') 
add_napari_layer(adata, column = 'TraesCS4D03G0546300') 
add_napari_layer(adata, column = 'TraesCS7D03G0517200') 
add_napari_layer(adata, column = 'TraesCS3D03G0079700') 
add_napari_layer(adata, column = 'TraesCS3D03G0946200') 
add_napari_layer(adata, column = 'TraesCS4B03G0618200') 
add_napari_layer(adata, column = 'TraesCS3D03G0238700') 
add_napari_layer(adata, column = 'TraesCS3D03G0079900') 
add_napari_layer(adata, column = 'TraesCS6B03G0962400') 
add_napari_layer(adata, column = 'TraesCS1D03G0681300') 
add_napari_layer(adata, column = 'TraesCS3B03G0311700') 
add_napari_layer(adata, column = 'TraesCS3D03G0079600') 
add_napari_layer(adata, column = 'TraesCS1A03G0193000') 
add_napari_layer(adata, column = 'TraesCS7B03G0223000') 
add_napari_layer(adata, column = 'TraesCS7B03G0355800') 
add_napari_layer(adata, column = 'TraesCS1B03G0260800') 
add_napari_layer(adata, column = 'TraesCS7B03G0170600') 
add_napari_layer(adata, column = 'TraesCS1B03G0819300') 
add_napari_layer(adata, column = 'TraesCS3D03G0079800') 













In [19]:
# cluster 6 procambium 
add_napari_layer(adata, column = 'TraesCS4D03G0691800') 
add_napari_layer(adata, column = 'TraesCS2B03G1372800') 
add_napari_layer(adata, column = 'TraesCS3A03G1137900') 
add_napari_layer(adata, column = 'TraesCS2D03G1158000') 
add_napari_layer(adata, column = 'TraesCS4A03G0786200') 
add_napari_layer(adata, column = 'TraesCS3B03G1323800') 
add_napari_layer(adata, column = 'TraesCS2A03G1202600') 
add_napari_layer(adata, column = 'TraesCS4D03G0264900LC') 
add_napari_layer(adata, column = 'TraesCS5D03G1199300') 
add_napari_layer(adata, column = 'TraesCS5A03G0127300') 
add_napari_layer(adata, column = 'TraesCS4A03G0535400') 
add_napari_layer(adata, column = 'TraesCS7B03G0588000') 
add_napari_layer(adata, column = 'TraesCS5D03G0150000LC') 
add_napari_layer(adata, column = 'TraesCS7D03G0728800') 
add_napari_layer(adata, column = 'TraesCS5B03G0138900') 
add_napari_layer(adata, column = 'TraesCS4B03G0238600') 
add_napari_layer(adata, column = 'TraesCS3A03G0715100') 
add_napari_layer(adata, column = 'TraesCS3B03G1338300') 
add_napari_layer(adata, column = 'TraesCS3B03G0820500') 
add_napari_layer(adata, column = 'TraesCS5B03G0547100') 
add_napari_layer(adata, column = 'TraesCS1D03G1000900') 
add_napari_layer(adata, column = 'TraesCS1B03G0187300') 
add_napari_layer(adata, column = 'TraesCS7A03G0757500') 
add_napari_layer(adata, column = 'TraesCS1B03G1236400') 
add_napari_layer(adata, column = 'TraesCS3D03G1070900') 
add_napari_layer(adata, column = 'TraesCS3B03G0250100') 
add_napari_layer(adata, column = 'TraesCS4D03G0223700') 
add_napari_layer(adata, column = 'TraesCS5B03G1121200') 
add_napari_layer(adata, column = 'TraesCS6B03G0419800') 
add_napari_layer(adata, column = 'TraesCS4B03G0262700') 





















In [20]:
# cluster 7 epidermis 
add_napari_layer(adata, column = 'TraesCS1D03G0362900') 
add_napari_layer(adata, column = 'TraesCS3B03G1278300') 
add_napari_layer(adata, column = 'TraesCS7A03G1125900') 
add_napari_layer(adata, column = 'TraesCS4B03G0721400') 
add_napari_layer(adata, column = 'TraesCS2B03G0426900') 
add_napari_layer(adata, column = 'TraesCS2B03G0426800') 
add_napari_layer(adata, column = 'TraesCS3D03G1015700') 
add_napari_layer(adata, column = 'TraesCS4A03G0065400') 
add_napari_layer(adata, column = 'TraesCS7A03G0992300LC') 
add_napari_layer(adata, column = 'TraesCS6B03G0242600') 
add_napari_layer(adata, column = 'TraesCS2B03G1540400') 
add_napari_layer(adata, column = 'TraesCS2A03G0309200') 
add_napari_layer(adata, column = 'TraesCS2B03G1534500') 
add_napari_layer(adata, column = 'TraesCS2B03G0426600LC') 
add_napari_layer(adata, column = 'TraesCS3D03G0000500') 
add_napari_layer(adata, column = 'TraesCS2A03G0309100LC') 
add_napari_layer(adata, column = 'TraesCS7B03G0829700LC') 
add_napari_layer(adata, column = 'TraesCS3A03G1099100') 
add_napari_layer(adata, column = 'TraesCS5B03G1313300') 
add_napari_layer(adata, column = 'TraesCS7D03G0949000') 
add_napari_layer(adata, column = 'TraesCS2D03G0324400') 
add_napari_layer(adata, column = 'TraesCS4A03G0828700') 
add_napari_layer(adata, column = 'TraesCS7A03G0992500LC') 
add_napari_layer(adata, column = 'TraesCS7A03G0992200LC') 
add_napari_layer(adata, column = 'TraesCS3B03G0134900') 
add_napari_layer(adata, column = 'TraesCS3A03G0103100') 
add_napari_layer(adata, column = 'TraesCS7D03G0969900LC') 
add_napari_layer(adata, column = 'TraesCS2D03G0324500') 
add_napari_layer(adata, column = 'TraesCS5D03G1173000') 














In [21]:
# cluster 8  no obvious pattern   condense cluster 1 7 8 into one 
add_napari_layer(adata, column = 'TraesCS2A03G0295100') 
add_napari_layer(adata, column = 'TraesCS2D03G0929500') 
add_napari_layer(adata, column = 'TraesCS1A03G0534900') 
add_napari_layer(adata, column = 'TraesCS2B03G1100500') 
add_napari_layer(adata, column = 'TraesCS1B03G0625100') 
add_napari_layer(adata, column = 'TraesCS7D03G1266700') 
add_napari_layer(adata, column = 'TraesCS4A03G0625000') 
add_napari_layer(adata, column = 'TraesCS7A03G0045400') 
add_napari_layer(adata, column = 'TraesCS4B03G0112400') 
add_napari_layer(adata, column = 'TraesCS3D03G0133700') 
add_napari_layer(adata, column = 'TraesCS2A03G0991200') 
add_napari_layer(adata, column = 'TraesCS2D03G0321900') 
add_napari_layer(adata, column = 'TraesCS5A03G0951600') 
add_napari_layer(adata, column = 'TraesCS6B03G1299400') 
add_napari_layer(adata, column = 'TraesCS2A03G0821300') 
add_napari_layer(adata, column = 'TraesCS7D03G0043000') 
add_napari_layer(adata, column = 'TraesCS5D03G0906800') 
add_napari_layer(adata, column = 'TraesCS2A03G1162700') 
add_napari_layer(adata, column = 'TraesCS1B03G0637100') 
add_napari_layer(adata, column = 'TraesCS2D03G0496100') 








In [22]:
# cluster 9 endodermis
add_napari_layer(adata, column = 'TraesCS6D03G0764800') 
add_napari_layer(adata, column = 'TraesCS6A03G0889900') 
add_napari_layer(adata, column = 'TraesCS5A03G0962700') 
add_napari_layer(adata, column = 'TraesCS5D03G0492700') 
add_napari_layer(adata, column = 'TraesCS5B03G0538700') 
add_napari_layer(adata, column = 'TraesCS7A03G1365400') 
add_napari_layer(adata, column = 'TraesCS3D03G0720200') 
add_napari_layer(adata, column = 'TraesCS3B03G0898100') 
add_napari_layer(adata, column = 'TraesCS1B03G0907000') 
add_napari_layer(adata, column = 'TraesCS1B03G0907000') 
add_napari_layer(adata, column = 'TraesCS2D03G0692900') 
add_napari_layer(adata, column = 'TraesCS6A03G0661900') 
add_napari_layer(adata, column = 'TraesCS7B03G0218800') 
add_napari_layer(adata, column = 'TraesCS7D03G0401600') 
add_napari_layer(adata, column = 'TraesCS2B03G0423100') 
add_napari_layer(adata, column = 'TraesCS1D03G0105400') 
add_napari_layer(adata, column = 'TraesCS3A03G0429700') 
add_napari_layer(adata, column = 'TraesCS3D03G0402600') 
add_napari_layer(adata, column = 'TraesCS3B03G0508400') 
add_napari_layer(adata, column = 'TraesCS2D03G0321300') 
















layer reused


In [23]:
# cluster 9 endodermis 
add_napari_layer(adata, column = 'TraesCS6B03G1068700') 
add_napari_layer(adata, column = 'TraesCS5D03G0917200') 
add_napari_layer(adata, column = 'TraesCS5B03G1013000') 
add_napari_layer(adata, column = 'TraesCS3A03G0779600') 
add_napari_layer(adata, column = 'TraesCS5A03G0526900') 
add_napari_layer(adata, column = 'TraesCS1D03G0995400') 
add_napari_layer(adata, column = 'TraesCS1A03G0789700') 
add_napari_layer(adata, column = 'TraesCS7B03G1305600') 
add_napari_layer(adata, column = 'TraesCS2A03G0303400') 
add_napari_layer(adata, column = 'TraesCS7D03G1292400') 
add_napari_layer(adata, column = 'TraesCS2B03G0825500') 
add_napari_layer(adata, column = 'TraesCS1B03G0906900') 
add_napari_layer(adata, column = 'TraesCS3D03G0402300') 
add_napari_layer(adata, column = 'TraesCS3D03G0402800') 
add_napari_layer(adata, column = 'TraesCS6B03G0815900') 
add_napari_layer(adata, column = 'TraesCS3B03G0508500') 
add_napari_layer(adata, column = 'TraesCS7A03G0413900') 
add_napari_layer(adata, column = 'TraesCS1A03G0117100') 
add_napari_layer(adata, column = 'TraesCS6D03G0894200') 
add_napari_layer(adata, column = 'TraesCS1B03G0155900') 












In [24]:
# cluster 10 xylem
add_napari_layer(adata, column = 'TraesCS7D03G1014000') 
add_napari_layer(adata, column = 'TraesCS4A03G0812200') 
add_napari_layer(adata, column = 'TraesCS2D03G0379800') 
add_napari_layer(adata, column = 'TraesCS1A03G0223500') 
add_napari_layer(adata, column = 'TraesCS1B03G0493300') 
add_napari_layer(adata, column = 'TraesCS2B03G0810100') 
add_napari_layer(adata, column = 'TraesCS5A03G0570800') 
add_napari_layer(adata, column = 'TraesCS3A03G0722200') 
add_napari_layer(adata, column = 'TraesCS2D03G0676600') 
add_napari_layer(adata, column = 'TraesCS7D03G0927700') 
add_napari_layer(adata, column = 'TraesCS2D03G0060400') 
add_napari_layer(adata, column = 'TraesCS5B03G0582100') 
add_napari_layer(adata, column = 'TraesCS1D03G0236500') 
add_napari_layer(adata, column = 'TraesCS1B03G0066500') 
add_napari_layer(adata, column = 'TraesCS2A03G0734000') 
add_napari_layer(adata, column = 'TraesCS2D03G0673900') 
add_napari_layer(adata, column = 'TraesCSU03G0020100') 
add_napari_layer(adata, column = 'TraesCS3D03G0666000') 
add_napari_layer(adata, column = 'TraesCS1D03G0371000') 
add_napari_layer(adata, column = 'TraesCS1B03G0325900') 
add_napari_layer(adata, column = 'TraesCS5D03G0536900') 
add_napari_layer(adata, column = 'TraesCS7B03G1041300') 
add_napari_layer(adata, column = 'TraesCS3A03G0987500') 
add_napari_layer(adata, column = 'TraesCS2D03G0381300') 
add_napari_layer(adata, column = 'TraesCS2A03G0364600') 
add_napari_layer(adata, column = 'TraesCS2B03G0806600') 
add_napari_layer(adata, column = 'TraesCS1A03G0402200') 
add_napari_layer(adata, column = 'TraesCS6A03G0080100') 
add_napari_layer(adata, column = 'TraesCS4B03G0486500') 
add_napari_layer(adata, column = 'TraesCS3D03G0008300') 
add_napari_layer(adata, column = 'TraesCS1B03G0516300') 
add_napari_layer(adata, column = 'TraesCS2A03G0730800') 
add_napari_layer(adata, column = 'TraesCS3B03G0012900') 
add_napari_layer(adata, column = 'TraesCS1D03G0402900') 
add_napari_layer(adata, column = 'TraesCSU03G0019900') 
add_napari_layer(adata, column = 'TraesCS4A03G0280400') 
add_napari_layer(adata, column = 'TraesCS3B03G1344800') 
add_napari_layer(adata, column = 'TraesCS7D03G0430200') 
add_napari_layer(adata, column = 'TraesCS2B03G0484900') 
add_napari_layer(adata, column = 'TraesCS4D03G0434100') 
add_napari_layer(adata, column = 'TraesCS7A03G0446200') 
add_napari_layer(adata, column = 'TraesCS7A03G0969300') 
add_napari_layer(adata, column = 'TraesCS7B03G0805100') 
add_napari_layer(adata, column = 'TraesCS6D03G0082600') 
add_napari_layer(adata, column = 'TraesCS6D03G0654300') 
add_napari_layer(adata, column = 'TraesCS1D03G0049400') 
add_napari_layer(adata, column = 'TraesCS1A03G0062800') 
add_napari_layer(adata, column = 'TraesCS5B03G0387000') 
add_napari_layer(adata, column = 'TraesCS5A03G0388600') 
add_napari_layer(adata, column = 'TraesCS5D03G0377000') 




In [25]:
# cluster 11  something close to cluster 4 provascular cells  should be merged 
add_napari_layer(adata, column = 'TraesCS7B03G0003800') 
add_napari_layer(adata, column = 'TraesCS2A03G0775400') 
add_napari_layer(adata, column = 'TraesCS2A03G0180600') 
add_napari_layer(adata, column = 'TraesCS4A03G1032000') 
add_napari_layer(adata, column = 'TraesCS6A03G0965900') 
add_napari_layer(adata, column = 'TraesCS7D03G1266100') 
add_napari_layer(adata, column = 'TraesCS2D03G0715400') 
add_napari_layer(adata, column = 'TraesCS2D03G0958800') 
add_napari_layer(adata, column = 'TraesCS6D03G0845500') 
add_napari_layer(adata, column = 'TraesCS7D03G0146000') 
add_napari_layer(adata, column = 'TraesCS2B03G0856800') 
add_napari_layer(adata, column = 'TraesCS5D03G1233700') 
add_napari_layer(adata, column = 'TraesCS5B03G1379500') 
add_napari_layer(adata, column = 'TraesCS3B03G0850100') 
add_napari_layer(adata, column = 'TraesCS3A03G0744700') 
add_napari_layer(adata, column = 'TraesCS3D03G0686200') 
add_napari_layer(adata, column = 'TraesCS2D03G0276000LC') 
add_napari_layer(adata, column = 'TraesCS2B03G0368000') 
add_napari_layer(adata, column = 'TraesCS1B03G0711800') 
add_napari_layer(adata, column = 'TraesCS4D03G0661400') 














In [26]:
# cluster 12 dividing cells
add_napari_layer(adata, column = 'TraesCS1A03G0768800') 
add_napari_layer(adata, column = 'TraesCS7B03G0367600') 
add_napari_layer(adata, column = 'TraesCS5B03G1032700') 
add_napari_layer(adata, column = 'TraesCS5D03G0934500') 
add_napari_layer(adata, column = 'TraesCS5A03G0986700') 
add_napari_layer(adata, column = 'TraesCS3B03G1062100') 
add_napari_layer(adata, column = 'TraesCS3A03G0732200LC') 
add_napari_layer(adata, column = 'TraesCS1B03G0879500') 
add_napari_layer(adata, column = 'TraesCS7D03G1262100') 
add_napari_layer(adata, column = 'TraesCS1D03G0508400') 
add_napari_layer(adata, column = 'TraesCS1D03G0735800') 
add_napari_layer(adata, column = 'TraesCS7A03G1335600') 
add_napari_layer(adata, column = 'TraesCS1B03G0616100') 
add_napari_layer(adata, column = 'TraesCS3A03G0935100') 
add_napari_layer(adata, column = 'TraesCS2A03G0132400') 
add_napari_layer(adata, column = 'TraesCS3D03G0969300') 
add_napari_layer(adata, column = 'TraesCS7B03G1269700') 
add_napari_layer(adata, column = 'TraesCS1A03G0526900') 
add_napari_layer(adata, column = 'TraesCS3B03G1202600') 
add_napari_layer(adata, column = 'TraesCS4D03G0742700') 
add_napari_layer(adata, column = 'TraesCS3B03G0918400') 
add_napari_layer(adata, column = 'TraesCS4B03G0833500') 
add_napari_layer(adata, column = 'TraesCS3D03G0735900') 
add_napari_layer(adata, column = 'TraesCS2D03G0129500') 
add_napari_layer(adata, column = 'TraesCS3A03G0796600') 
add_napari_layer(adata, column = 'TraesCS2B03G0955500') 
add_napari_layer(adata, column = 'TraesCS2A03G0863000') 
add_napari_layer(adata, column = 'TraesCS3B03G0839200LC') 
add_napari_layer(adata, column = 'TraesCS2D03G0803000') 


















In [11]:
# cluster 13 
add_napari_layer(adata, column = 'TraesCS2B03G0108200') 
add_napari_layer(adata, column = 'TraesCS2D03G0068900') 
add_napari_layer(adata, column = 'TraesCS3B03G0900500') 
add_napari_layer(adata, column = 'TraesCS2A03G0073400') 
add_napari_layer(adata, column = 'TraesCS3D03G0721900') 
add_napari_layer(adata, column = 'TraesCS3A03G0782300') 
add_napari_layer(adata, column = 'TraesCS1D03G0884600') 
add_napari_layer(adata, column = 'TraesCS3D03G0599200') 
add_napari_layer(adata, column = 'TraesCS6A03G0666600') 
add_napari_layer(adata, column = 'TraesCS3A03G0654700') 
add_napari_layer(adata, column = 'TraesCS1D03G0605000') 
add_napari_layer(adata, column = 'TraesCS1B03G0733300') 
add_napari_layer(adata, column = 'TraesCS1A03G0647000') 
add_napari_layer(adata, column = 'TraesCS1A03G0906900') 
add_napari_layer(adata, column = 'TraesCS3B03G0742300') 
add_napari_layer(adata, column = 'TraesCS6A03G0202900') 
add_napari_layer(adata, column = 'TraesCS4D03G0020800') 
add_napari_layer(adata, column = 'TraesCS7D03G0500500') 
add_napari_layer(adata, column = 'TraesCS5D03G0557600') 
add_napari_layer(adata, column = 'TraesCS3B03G0486000') 









In [12]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS6B03G0810000') 
add_napari_layer(adata, column = 'TraesCS5A03G0536500') 
add_napari_layer(adata, column = 'TraesCS6B03G0398800') 
add_napari_layer(adata, column = 'TraesCS6A03G0306800') 
add_napari_layer(adata, column = 'TraesCS4D03G0760800') 
add_napari_layer(adata, column = 'TraesCS7B03G0335800') 
add_napari_layer(adata, column = 'TraesCS4B03G0030200') 
add_napari_layer(adata, column = 'TraesCS6B03G0270900') 
add_napari_layer(adata, column = 'TraesCS3A03G1157200') 
add_napari_layer(adata, column = 'TraesCS5A03G0819000') 
add_napari_layer(adata, column = 'TraesCS3B03G1380200') 
add_napari_layer(adata, column = 'TraesCS4A03G0754700') 
add_napari_layer(adata, column = 'TraesCS3B03G0494600') 
add_napari_layer(adata, column = 'TraesCS4B03G0725300') 
add_napari_layer(adata, column = 'TraesCS3A03G0407400') 
add_napari_layer(adata, column = 'TraesCS6B03G0419400') 
add_napari_layer(adata, column = 'TraesCS1D03G0349000') 
add_napari_layer(adata, column = 'TraesCS3A03G0709800') 
add_napari_layer(adata, column = 'TraesCS3D03G0382200') 
add_napari_layer(adata, column = 'TraesCS4D03G0091800') 



In [13]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS3D03G1105300') 
add_napari_layer(adata, column = 'TraesCS1A03G0542800') 
add_napari_layer(adata, column = 'TraesCS3A03G0390900') 
add_napari_layer(adata, column = 'TraesCS1B03G0715800LC') 
add_napari_layer(adata, column = 'TraesCS1D03G0587800') 
add_napari_layer(adata, column = 'TraesCS4A03G0687000') 
add_napari_layer(adata, column = 'TraesCS6D03G0279900') 
add_napari_layer(adata, column = 'TraesCS1A03G0361700') 
add_napari_layer(adata, column = 'TraesCS1B03G0450900') 
add_napari_layer(adata, column = 'TraesCS4B03G0108300') 
add_napari_layer(adata, column = 'TraesCS1A03G0626900') 
add_napari_layer(adata, column = 'TraesCS5B03G0548000') 
add_napari_layer(adata, column = 'TraesCS5A03G1176600LC') 
add_napari_layer(adata, column = 'TraesCS7B03G0047600') 
add_napari_layer(adata, column = 'TraesCS2D03G0137900') 
add_napari_layer(adata, column = 'TraesCS4A03G0321300') 
add_napari_layer(adata, column = 'TraesCS5D03G0143700') 
add_napari_layer(adata, column = 'TraesCS2A03G0138900') 
add_napari_layer(adata, column = 'TraesCS5A03G1176700') 
add_napari_layer(adata, column = 'TraesCS3B03G0001000') 




In [14]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS3A03G0589600') 
add_napari_layer(adata, column = 'TraesCS5D03G0500000') 
add_napari_layer(adata, column = 'TraesCS2B03G0197500') 
add_napari_layer(adata, column = 'TraesCS1A03G0652600') 
add_napari_layer(adata, column = 'TraesCS5B03G0133000') 
add_napari_layer(adata, column = 'TraesCS7D03G0337200') 
add_napari_layer(adata, column = 'TraesCSU03G0052100') 
add_napari_layer(adata, column = 'TraesCS4D03G0761000') 
add_napari_layer(adata, column = 'TraesCS6D03G0676700') 
add_napari_layer(adata, column = 'TraesCS5D03G0156400') 
add_napari_layer(adata, column = 'TraesCS5D03G0993000') 
add_napari_layer(adata, column = 'TraesCS4B03G0725100') 
add_napari_layer(adata, column = 'TraesCS7A03G0853800') 
add_napari_layer(adata, column = 'TraesCS4D03G0645400') 
add_napari_layer(adata, column = 'TraesCS7D03G0567100') 
add_napari_layer(adata, column = 'TraesCS6B03G0482100') 
add_napari_layer(adata, column = 'TraesCS6D03G0555500') 
add_napari_layer(adata, column = 'TraesCS1D03G0173900') 
add_napari_layer(adata, column = 'TraesCS3D03G0090600') 
add_napari_layer(adata, column = 'TraesCS5A03G1177000') 

In [15]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS7D03G0264400') 
add_napari_layer(adata, column = 'TraesCS4A03G0061200LC') 
add_napari_layer(adata, column = 'TraesCS2A03G1339100') 
add_napari_layer(adata, column = 'TraesCS1B03G0015300') 
add_napari_layer(adata, column = 'TraesCS1A03G0916900') 
add_napari_layer(adata, column = 'TraesCS2D03G1309000') 
add_napari_layer(adata, column = 'TraesCS7B03G0314400') 
add_napari_layer(adata, column = 'TraesCS5D03G0499900') 
add_napari_layer(adata, column = 'TraesCS1D03G0528800') 
add_napari_layer(adata, column = 'TraesCS1B03G0653700') 
add_napari_layer(adata, column = 'TraesCS7A03G0285400') 
add_napari_layer(adata, column = 'TraesCS2A03G0466400') 
add_napari_layer(adata, column = 'TraesCS5A03G1054700') 
add_napari_layer(adata, column = 'TraesCS2A03G1034400') 
add_napari_layer(adata, column = 'TraesCS2D03G0978200') 
add_napari_layer(adata, column = 'TraesCS2D03G0915700') 
add_napari_layer(adata, column = 'TraesCS5B03G0852500') 
add_napari_layer(adata, column = 'TraesCSU03G0248400') 
add_napari_layer(adata, column = 'TraesCS6B03G0945800') 
add_napari_layer(adata, column = 'TraesCS1B03G1177400') 



In [16]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS1B03G1064900') 
add_napari_layer(adata, column = 'TraesCS4D03G0645300') 
add_napari_layer(adata, column = 'TraesCS6A03G0762000') 
add_napari_layer(adata, column = 'TraesCS4B03G0329500') 
add_napari_layer(adata, column = 'TraesCS1B03G0740500') 
add_napari_layer(adata, column = 'TraesCS2B03G1081500') 
add_napari_layer(adata, column = 'TraesCS5A03G1214500') 
add_napari_layer(adata, column = 'TraesCS7A03G0988200') 
add_napari_layer(adata, column = 'TraesCS7D03G0820700') 
add_napari_layer(adata, column = 'TraesCS7D03G0865700') 
add_napari_layer(adata, column = 'TraesCS1A03G0015600') 
add_napari_layer(adata, column = 'TraesCS2A03G1116900') 
add_napari_layer(adata, column = 'TraesCS2B03G1570200') 
add_napari_layer(adata, column = 'TraesCS3A03G0103400') 
add_napari_layer(adata, column = 'TraesCS7B03G0825600') 
add_napari_layer(adata, column = 'TraesCS5A03G0133800') 
add_napari_layer(adata, column = 'TraesCS4D03G0534000') 
add_napari_layer(adata, column = 'TraesCS1A03G0658100') 
add_napari_layer(adata, column = 'TraesCS1D03G1022100') 
add_napari_layer(adata, column = 'TraesCS2A03G0976200') 

In [17]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS5B03G1350700') 
add_napari_layer(adata, column = 'TraesCS1B03G1257600') 
add_napari_layer(adata, column = 'TraesCS1B03G0245300') 
add_napari_layer(adata, column = 'TraesCS7D03G1119700') 
add_napari_layer(adata, column = 'TraesCS5D03G0647900') 
add_napari_layer(adata, column = 'TraesCS4B03G0030100') 
add_napari_layer(adata, column = 'TraesCS7B03G1048700') 
add_napari_layer(adata, column = 'TraesCS6D03G0278900') 
add_napari_layer(adata, column = 'TraesCS2B03G1160800') 
add_napari_layer(adata, column = 'TraesCS7D03G0783400') 
add_napari_layer(adata, column = 'TraesCS5B03G1104600') 
add_napari_layer(adata, column = 'TraesCS4D03G0801600') 
add_napari_layer(adata, column = 'TraesCS1A03G0178500') 
add_napari_layer(adata, column = 'TraesCS4B03G0725200') 
add_napari_layer(adata, column = 'TraesCS3A03G0666400') 
add_napari_layer(adata, column = 'TraesCS5A03G0121700') 
add_napari_layer(adata, column = 'TraesCS7B03G1048800') 
add_napari_layer(adata, column = 'TraesCS7A03G0498400') 
add_napari_layer(adata, column = 'TraesCS5B03G0158100') 
add_napari_layer(adata, column = 'TraesCS5A03G1287500') 



In [18]:
# cluster 13
add_napari_layer(adata, column = 'TraesCS5D03G0451100') 
add_napari_layer(adata, column = 'TraesCS5D03G0780700') 
add_napari_layer(adata, column = 'TraesCS5B03G1381100') 
add_napari_layer(adata, column = 'TraesCS3B03G0761200') 
add_napari_layer(adata, column = 'TraesCS1A03G0548800') 
add_napari_layer(adata, column = 'TraesCS5B03G0609100') 
add_napari_layer(adata, column = 'TraesCS3A03G0413300') 
add_napari_layer(adata, column = 'TraesCS1B03G1081000') 
add_napari_layer(adata, column = 'TraesCS7B03G0218400') 
add_napari_layer(adata, column = 'TraesCS5A03G0594800') 

In [19]:
# Cluster  14
add_napari_layer(adata, column = 'TraesCS7D03G0226600') 
add_napari_layer(adata, column = 'TraesCS3D03G0697200') 
add_napari_layer(adata, column = 'TraesCS2D03G0971100') 
add_napari_layer(adata, column = 'TraesCS3D03G0382600') 
add_napari_layer(adata, column = 'TraesCS2A03G1032300') 
add_napari_layer(adata, column = 'TraesCS3B03G0864600') 
add_napari_layer(adata, column = 'TraesCS1D03G0043400') 
add_napari_layer(adata, column = 'TraesCS3D03G1163600') 
add_napari_layer(adata, column = 'TraesCS3B03G1268400') 
add_napari_layer(adata, column = 'TraesCS3D03G1163500') 
add_napari_layer(adata, column = 'TraesCS3A03G0756500') 
add_napari_layer(adata, column = 'TraesCS1A03G0053300') 
add_napari_layer(adata, column = 'TraesCS3B03G1464600') 
add_napari_layer(adata, column = 'TraesCS1B03G0058000') 
add_napari_layer(adata, column = 'TraesCS3B03G1464500') 
add_napari_layer(adata, column = 'TraesCS2B03G1153200') 
add_napari_layer(adata, column = 'TraesCS2D03G0971700') 
add_napari_layer(adata, column = 'TraesCS3A03G1224300') 
add_napari_layer(adata, column = 'TraesCS2D03G0971800') 
add_napari_layer(adata, column = 'TraesCS3D03G1163700') 

















In [20]:
# Cluster 14
add_napari_layer(adata, column = 'TraesCS2B03G1399100') 
add_napari_layer(adata, column = 'TraesCS6B03G0167500') 
add_napari_layer(adata, column = 'TraesCS2B03G1214600') 
add_napari_layer(adata, column = 'TraesCS3A03G1226700') 
add_napari_layer(adata, column = 'TraesCS3D03G0655200') 
add_napari_layer(adata, column = 'TraesCS2A03G1032100') 
add_napari_layer(adata, column = 'TraesCS2D03G1025100') 
add_napari_layer(adata, column = 'TraesCS3A03G1224100') 
add_napari_layer(adata, column = 'TraesCS6A03G0118600') 
add_napari_layer(adata, column = 'TraesCS2A03G1083300') 
add_napari_layer(adata, column = 'TraesCS3B03G0460700') 
add_napari_layer(adata, column = 'TraesCS3A03G0390700') 
add_napari_layer(adata, column = 'TraesCS3B03G1468100') 
add_napari_layer(adata, column = 'TraesCS3B03G1468200') 
add_napari_layer(adata, column = 'TraesCS3D03G0361400') 
add_napari_layer(adata, column = 'TraesCS7A03G0910800') 
add_napari_layer(adata, column = 'TraesCS5D03G0905800') 
add_napari_layer(adata, column = 'TraesCS4D03G0755100') 
add_napari_layer(adata, column = 'TraesCS2A03G1031900') 



In [38]:
# Cortex
add_napari_layer(adata, column = 'TraesCS3B03G1120300')  # Cortex 1
add_napari_layer(adata, column = 'TraesCS2B03G1501600')  # Cortex 2

/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/yujike/opt/anaconda3/envs/Pandas/lib/python3.8/site-packages/napari/_vispy/layers/image.py:259: UserWarning: data shape (26450, 26459) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [ ]:
# Xylem
add_napari_layer(adata, column = 'TraesCS5D03G0377000')  # Xylem 1
add_napari_layer(adata, column = 'TraesCS5B03G0387000')  # Xylem 2
add_napari_layer(adata, column = 'TraesCS1A03G0062800')  # Xylem 3

In [ ]:
# Phloem
add_napari_layer(adata, column = 'TraesCS6A03G0459900')  # Phloem 1

In [ ]:
# Pericycle
add_napari_layer(adata, column = 'TraesCS4D03G0748700')  # Pericycle 1
add_napari_layer(adata, column = 'TraesCS4B03G0840800')  # Pericycle 1

In [ ]:
# Endodermis
add_napari_layer(adata, column = 'TraesCS7B03G0854100')  # Endodermis 1

In [ ]:
# Epidermis
add_napari_layer(adata, column = 'TraesCS2B03G0426800')  # Epidermis 1
add_napari_layer(adata, column = 'TraesCS2B03G0426900')  # Epidermis 2
add_napari_layer(adata, column = 'TraesCS2D03G0324500')  # Epidermis 3
add_napari_layer(adata, column = 'TraesCS5D03G1173000')  # Epidermis 4
add_napari_layer(adata, column = 'TraesCS7B03G1286700')  # Epidermis 5

In [ ]:
# House-keeping genes
add_napari_layer(adata, column = 'TraesCS1B03G0048800')  # Actin 1
add_napari_layer(adata, column = 'TraesCS1D03G0038800')  # Actin 2

In [14]:
# Provascular cells
add_napari_layer(adata, column = 'TraesCS3A03G0016400') # WAT1
add_napari_layer(adata, column = 'TraesCS1B03G0051800') # WAT1
add_napari_layer(adata, column = 'TraesCS3B03G0036500') # WAT1
add_napari_layer(adata, column = 'TraesCS3D03G0013700') # WAT1
add_napari_layer(adata, column = 'TraesCS1B03G0053600') # WAT1


layer reused


In [39]:
# S phase  
add_napari_layer(adata, column = 'TraesCS1A03G0879600') # 
add_napari_layer(adata, column = 'TraesCS1A03G0879000') # 
add_napari_layer(adata, column = 'TraesCS1D03G0853100') # 
add_napari_layer(adata, column = 'TraesCS3A03G0316900') # 
add_napari_layer(adata, column = 'TraesCS1A03G0723300') # 



In [1]:
# Read the contents of the input file
with open('/Users/yujike/Desktop/Molecular_Plant_2023/cross-species/rice/Zhang_rice/MINI-EX2.0/SCT_count_matrix.txt', 'r') as infile:
    lines = infile.readlines()

# Create a dictionary to track lines based on their first column value
seen_first_column = {}

# Collect unique lines while keeping one instance of each value
unique_lines = []
for line in lines:
    columns = line.strip().split('\t')  # Assuming columns are tab-separated
    first_column_value = columns[0]
    
    if first_column_value not in seen_first_column:
        unique_lines.append(line)
        seen_first_column[first_column_value] = True

# Write the unique lines back to the file
with open('/Users/yujike/Desktop/Molecular_Plant_2023/cross-species/rice/Zhang_rice/MINI-EX2.0/SCT_count_matrix.txt', 'w') as outfile:
    outfile.writelines(unique_lines)
    